# Notebook de extracción de datos de diversas páginas web

In [ ]:
# Importo las librerias
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
# Primera url
url_1="https://www.carwow.es/suvs/7-plazas?utm_source=google&utm_medium=cpc&utm_account=1067952725&utm_campaign=16616870776&utm_group=140354601048&utm_keyword=&device=c&campaignid=16616870776&adgroupid=140354601048&gad_source=1&gbraid=0AAAAADkNbBUPhyVGM4sZxbLeGiQKHZ1nw&gclid=CjwKCAjwq7fABhB2EiwAwk-YbDwojWaxN2WG0_cnvz9JYpB20AtA5WJy8TP3udES_xm-XQdCLAp3QRoCxhMQAvD_BwE"

In [3]:
# Genero conexion, compruebo el estado, y creo la sopa
conexion = requests.get(url_1)
print(conexion.status_code, conexion.reason)
sopa = bs(conexion.text, "lxml")

200 OK


In [4]:
# Genero el nombre de las listas que van a ir al DF, en función de los datos que me interesan a priori 
# con ayuda de la info de las webs
lista_coches_DF = []
lista_valoraciones_DF = []
lista_autonomía_DF = []
lista_a_favor_DF = []
lista_en_contra_DF =[]
lista_opinion_DF = []
lista_precios_DF = []

In [5]:
# Obtención de la primera lista
coches = sopa.find_all( 'h3', class_='deal-review-highlight-automated-ctas__title')
lista_coches = []
for indice, coche in enumerate(coches):
    lista_coches.append(coche.text.strip())
    indice += 1
print(lista_coches)

['1. Mazda CX-80', '2. Renault Espace', '3. Hyundai Santa Fe', '4. Skoda Kodiaq', '5. Peugeot 5008', '6. Volkswagen Tayron', '7. KIA Sorento', '8. Nissan X-Trail', '9. Mercedes-Benz GLB', '10. Toyota Highlander']


In [6]:
# Agrego la lista obtenida a la del DF. Lo hago así para no tener problemas y equivocaciones. 
# Se podría añadir directamente, pero en el futuro he tenido problemas en la obtención de datos incompletos y de esta manera agrego los datos ya completos
lista_coches_DF.extend(lista_coches)
lista_coches_DF

['1. Mazda CX-80',
 '2. Renault Espace',
 '3. Hyundai Santa Fe',
 '4. Skoda Kodiaq',
 '5. Peugeot 5008',
 '6. Volkswagen Tayron',
 '7. KIA Sorento',
 '8. Nissan X-Trail',
 '9. Mercedes-Benz GLB',
 '10. Toyota Highlander']

In [7]:
# Segundos datos
autonomía_eléctrico = sopa.find_all('div', class_='deal-review-highlight-automated-ctas__stat') 
info_adicional =[]
for indice, autonomia in enumerate(autonomía_eléctrico):
    info_adicional.append(autonomia.text.strip())
    indice += 1
print(info_adicional) # Unos tienen este dato y otros no, asi que ahora agregaré manualmente los datos nulos a la lista
    

['Autonomía homologada hasta 60 km', 'Autonomía homologada hasta 54 km', 'Autonomía homologada hasta 123 km', 'Autonomía homologada hasta 57 km']


In [8]:
# Como hay vehiculos que no tienen algún dato, voy a crear una función para poder añadir nulos o missing
#  en las posiciones que correspondan en cada lista y luego ya poder añadir las listas completas a las que van a forrmar el DF
def introducir_nulos(lista, posiciones):
    '''
    lista hace referencia a la lista a la que queremos añadir valores nulos
    posiciones lo introducimos como una tupla, con 1 ó varios valores según la necesidad
    '''
    for indice, posicion in enumerate(posiciones):
        lista.insert(posicion, None)
    print(lista)


In [9]:
# Función para generar listas completas de Nulos
def crear_lista_none(n):
  """
  Crea una lista de n elementos, todos con el valor None.
  """
  return [None] * n

In [10]:
# Añadir nulos en posiciones: 2, 4, 5, 8, 9, 10, ojo estás son las reales, en Python empiezan en 0
introducir_nulos(info_adicional, ( 1, 3, 4, 7, 8, 9))

['Autonomía homologada hasta 60 km', None, 'Autonomía homologada hasta 54 km', None, None, 'Autonomía homologada hasta 123 km', 'Autonomía homologada hasta 57 km', None, None, None]


In [11]:
# Tercera columna de informacion
cosas_a_favor= sopa.find_all('li', class_='list-icon__item list-icon__item--positive') # da valores repetidos porque están dos veces en el html
lista_a_favor = []
for indice, a_favor in enumerate(cosas_a_favor):
    lista_a_favor.append(a_favor.text.strip())
    indice +=1
print(lista_a_favor)

['Excelente eficiencia tanto en el PHEV como en el diésel', 'Fácil acceso y amplio espacio en la segunda y tercera fila de asientos', 'Alta calidad de construcción y acabados interiores', 'Excelente eficiencia tanto en el PHEV como en el diésel', 'Fácil acceso y amplio espacio en la segunda y tercera fila de asientos', 'Alta calidad de construcción y acabados interiores', 'Consumo', 'Equipamiento disponible', 'Opción de siete plazas sin coste', 'Consumo', 'Equipamiento disponible', 'Opción de siete plazas sin coste', 'Equipamiento y tecnología propios de coches de más alta gama', 'Motor híbrido potente y tracción total', 'Es un coche cómodo de conducir en toda circunstancia', 'Equipamiento y tecnología propios de coches de más alta gama', 'Motor híbrido potente y tracción total', 'Es un coche cómodo de conducir en toda circunstancia', 'Practicidad y espacio disponible', 'Oferta mecánica amplia', 'Confort de marcha', 'Practicidad y espacio disponible', 'Oferta mecánica amplia', 'Confort

In [12]:
# Genero una función para poder agrupar los valores ya que originalmente en la página vienen de 3 en tres
def agrupar_de_tres(lista): 
  '''Agrupa los elementos de una lista en sublistas de tamaño 3.

  Args:
    lista: La lista original.

  Returns:
    Una nueva lista que contiene sublistas de tamaño 3.
  '''
  return [lista[i:i+3] for i in range(0, len(lista), 3)]

In [13]:
lista_agrupada_a_favor = agrupar_de_tres(lista_a_favor)
lista_agrupada_a_favor

[['Excelente eficiencia tanto en el PHEV como en el diésel',
  'Fácil acceso y amplio espacio en la segunda y tercera fila de asientos',
  'Alta calidad de construcción y acabados interiores'],
 ['Excelente eficiencia tanto en el PHEV como en el diésel',
  'Fácil acceso y amplio espacio en la segunda y tercera fila de asientos',
  'Alta calidad de construcción y acabados interiores'],
 ['Consumo', 'Equipamiento disponible', 'Opción de siete plazas sin coste'],
 ['Consumo', 'Equipamiento disponible', 'Opción de siete plazas sin coste'],
 ['Equipamiento y tecnología propios de coches de más alta gama',
  'Motor híbrido potente y tracción total',
  'Es un coche cómodo de conducir en toda circunstancia'],
 ['Equipamiento y tecnología propios de coches de más alta gama',
  'Motor híbrido potente y tracción total',
  'Es un coche cómodo de conducir en toda circunstancia'],
 ['Practicidad y espacio disponible',
  'Oferta mecánica amplia',
  'Confort de marcha'],
 ['Practicidad y espacio dispo

In [14]:
# Para obtener sólo una copia de cada
lista_a_favor_corregida = [lista_agrupada_a_favor[i] for i in range(1, len(lista_agrupada_a_favor), 2)]
lista_a_favor_corregida

[['Excelente eficiencia tanto en el PHEV como en el diésel',
  'Fácil acceso y amplio espacio en la segunda y tercera fila de asientos',
  'Alta calidad de construcción y acabados interiores'],
 ['Consumo', 'Equipamiento disponible', 'Opción de siete plazas sin coste'],
 ['Equipamiento y tecnología propios de coches de más alta gama',
  'Motor híbrido potente y tracción total',
  'Es un coche cómodo de conducir en toda circunstancia'],
 ['Practicidad y espacio disponible',
  'Oferta mecánica amplia',
  'Confort de marcha'],
 ['Amplio interior con 7 plazas de serie',
  'Comodidad y refinamiento',
  'Motorizaciones híbridas y eléctricas, con buena autonomía'],
 ['Maletero muy grande', 'Aislamiento', 'Integración de ChatGPT'],
 ['Variedad de motores', 'Amplio y disponible con siete plazas', 'Precio'],
 ['Presentación vistosa',
  'Confort general',
  'Uno de los pocos de su tamaño con siete plazas'],
 ['Versatilidad', 'Diseño', 'Variedad'],
 ['Versatilidad', 'Facilidad de conducción', 'Eti

In [15]:
# La añado a la lista del DF
lista_a_favor_DF.extend(lista_a_favor_corregida)
lista_a_favor_DF

[['Excelente eficiencia tanto en el PHEV como en el diésel',
  'Fácil acceso y amplio espacio en la segunda y tercera fila de asientos',
  'Alta calidad de construcción y acabados interiores'],
 ['Consumo', 'Equipamiento disponible', 'Opción de siete plazas sin coste'],
 ['Equipamiento y tecnología propios de coches de más alta gama',
  'Motor híbrido potente y tracción total',
  'Es un coche cómodo de conducir en toda circunstancia'],
 ['Practicidad y espacio disponible',
  'Oferta mecánica amplia',
  'Confort de marcha'],
 ['Amplio interior con 7 plazas de serie',
  'Comodidad y refinamiento',
  'Motorizaciones híbridas y eléctricas, con buena autonomía'],
 ['Maletero muy grande', 'Aislamiento', 'Integración de ChatGPT'],
 ['Variedad de motores', 'Amplio y disponible con siete plazas', 'Precio'],
 ['Presentación vistosa',
  'Confort general',
  'Uno de los pocos de su tamaño con siete plazas'],
 ['Versatilidad', 'Diseño', 'Variedad'],
 ['Versatilidad', 'Facilidad de conducción', 'Eti

In [16]:
# Cuarta columna, pasa lo mismo que en el caso anterior
cosas_en_contra= sopa.find_all('li', class_='list-icon__item list-icon__item--negative') #valores repetidos de 3 en 3 
lista_en_contra = []
for indice, en_contra in enumerate(cosas_en_contra):
    lista_en_contra.append(en_contra.text.strip())
    indice += 1
print(lista_en_contra)

['La suspensión puede sentirse algo seca en ciertas superficies', 'Barqueo de la carrocería acusado en conducción alegre', 'Sistema multimedia algo limitado frente a algunos rivales', 'La suspensión puede sentirse algo seca en ciertas superficies', 'Barqueo de la carrocería acusado en conducción alegre', 'Sistema multimedia algo limitado frente a algunos rivales', 'Funcionamiento poco refinado en ocasiones', 'Gama reducida a una sola versión', 'Tacto del freno', 'Funcionamiento poco refinado en ocasiones', 'Gama reducida a una sola versión', 'Tacto del freno', 'Precio elevado', 'Escasa autonomía eléctrica de la versión híbrida enchufable', 'Alertas de los asistentes de conducción intrusivas y engorrosas de desconectar', 'Precio elevado', 'Escasa autonomía eléctrica de la versión híbrida enchufable', 'Alertas de los asistentes de conducción intrusivas y engorrosas de desconectar', 'Ergonomía de algunos mandos', 'Plazas de la tercera fila solo para niños', 'Motores diésel sin hibridación

In [17]:
# Agrupamos
lista_agrupada_en_contra = agrupar_de_tres(lista_en_contra)
lista_agrupada_en_contra

[['La suspensión puede sentirse algo seca en ciertas superficies',
  'Barqueo de la carrocería acusado en conducción alegre',
  'Sistema multimedia algo limitado frente a algunos rivales'],
 ['La suspensión puede sentirse algo seca en ciertas superficies',
  'Barqueo de la carrocería acusado en conducción alegre',
  'Sistema multimedia algo limitado frente a algunos rivales'],
 ['Funcionamiento poco refinado en ocasiones',
  'Gama reducida a una sola versión',
  'Tacto del freno'],
 ['Funcionamiento poco refinado en ocasiones',
  'Gama reducida a una sola versión',
  'Tacto del freno'],
 ['Precio elevado',
  'Escasa autonomía eléctrica de la versión híbrida enchufable',
  'Alertas de los asistentes de conducción intrusivas y engorrosas de desconectar'],
 ['Precio elevado',
  'Escasa autonomía eléctrica de la versión híbrida enchufable',
  'Alertas de los asistentes de conducción intrusivas y engorrosas de desconectar'],
 ['Ergonomía de algunos mandos',
  'Plazas de la tercera fila solo

In [18]:
# Para obtener sólo una copia
lista_en_contra_corregida = [lista_agrupada_en_contra[i] for i in range(1, len(lista_agrupada_en_contra), 2)]
print(lista_en_contra_corregida) 

[['La suspensión puede sentirse algo seca en ciertas superficies', 'Barqueo de la carrocería acusado en conducción alegre', 'Sistema multimedia algo limitado frente a algunos rivales'], ['Funcionamiento poco refinado en ocasiones', 'Gama reducida a una sola versión', 'Tacto del freno'], ['Precio elevado', 'Escasa autonomía eléctrica de la versión híbrida enchufable', 'Alertas de los asistentes de conducción intrusivas y engorrosas de desconectar'], ['Ergonomía de algunos mandos', 'Plazas de la tercera fila solo para niños', 'Motores diésel sin hibridación'], ['La calidad del sistema multimedia podría mejorar', 'El i-Cockpit seguirá sin convencer a algunos usuarios', 'Prestaciones'], ['PHEV solo de cinco plazas', 'La tercera fila de asientos puede quedar justa en espacio', 'Detalles de acabado'], ['Consumo', 'Mandos del climatizador', 'Prestaciones sólo correctas'], ['Rumorosidad en ciertas condiciones', 'Presencia de algunos plásticos duros', 'Manejo del selector de marchas'], ['Precio

In [19]:
# Añadimos a la del DF
lista_en_contra_DF.extend(lista_en_contra_corregida)
lista_en_contra_DF

[['La suspensión puede sentirse algo seca en ciertas superficies',
  'Barqueo de la carrocería acusado en conducción alegre',
  'Sistema multimedia algo limitado frente a algunos rivales'],
 ['Funcionamiento poco refinado en ocasiones',
  'Gama reducida a una sola versión',
  'Tacto del freno'],
 ['Precio elevado',
  'Escasa autonomía eléctrica de la versión híbrida enchufable',
  'Alertas de los asistentes de conducción intrusivas y engorrosas de desconectar'],
 ['Ergonomía de algunos mandos',
  'Plazas de la tercera fila solo para niños',
  'Motores diésel sin hibridación'],
 ['La calidad del sistema multimedia podría mejorar',
  'El i-Cockpit seguirá sin convencer a algunos usuarios',
  'Prestaciones'],
 ['PHEV solo de cinco plazas',
  'La tercera fila de asientos puede quedar justa en espacio',
  'Detalles de acabado'],
 ['Consumo', 'Mandos del climatizador', 'Prestaciones sólo correctas'],
 ['Rumorosidad en ciertas condiciones',
  'Presencia de algunos plásticos duros',
  'Manejo 

In [20]:
# Quinta columna
otras_opiniciones= sopa.find_all(class_='deal-review-highlight-automated-ctas__body--no-read-more')
lista_otras_opiniones = []
for índice, otra_opinion in enumerate(otras_opiniciones):
    lista_otras_opiniones.append(otra_opinion.text.strip())
    indice += 1
print(lista_otras_opiniones)

['El CX-80 usa la misma plataforma y motores que el CX-60 pero es 25 cm más largo, por lo que también destaca en tacto de conducción, la fuerza y elasticidad de su motor diésel de 6 cilindros con microhibridación y un interior con acabados de lujo. Todo ello por un precio muy razonable.', 'El nuevo Renault Espace es ahora un SUV con unas plazas traseras de la tercera fila con un espacio similar al de sus rivales, con una única motorización híbrida potente y bastante eficiente. Cuenta con lo último en tecnología multimedia, con una gran pantalla central y un software desarrollado por Google que funciona con fluidez, y tiene un chasis sofisticado con un buen comportamiento y confort.', 'El Santa Fe dispone de motorizaciones híbrida e híbrida enchufable y ahora es más lujoso y refinado. Tiene un habitáculo amplio con una buena calidad de acabados, su equipamiento es abundante y es muy confortable. Digamos que lo hace todo bien pero han subido el precio también bastante.', 'El Skoda Kodiaq

In [21]:
# Añadimos a la del DF
lista_opinion_DF.extend(lista_otras_opiniones)
lista_opinion_DF

['El CX-80 usa la misma plataforma y motores que el CX-60 pero es 25 cm más largo, por lo que también destaca en tacto de conducción, la fuerza y elasticidad de su motor diésel de 6 cilindros con microhibridación y un interior con acabados de lujo. Todo ello por un precio muy razonable.',
 'El nuevo Renault Espace es ahora un SUV con unas plazas traseras de la tercera fila con un espacio similar al de sus rivales, con una única motorización híbrida potente y bastante eficiente. Cuenta con lo último en tecnología multimedia, con una gran pantalla central y un software desarrollado por Google que funciona con fluidez, y tiene un chasis sofisticado con un buen comportamiento y confort.',
 'El Santa Fe dispone de motorizaciones híbrida e híbrida enchufable y ahora es más lujoso y refinado. Tiene un habitáculo amplio con una buena calidad de acabados, su equipamiento es abundante y es muy confortable. Digamos que lo hace todo bien pero han subido el precio también bastante.',
 'El Skoda Kod

In [22]:
# Siguiente columna
precios= sopa.find_all(class_='div tile-button__subtitle') # aparece dos veces cada dato en el html
lista_precios= []
for indice, precio in enumerate(precios):
    lista_precios.append(precio.text.strip())
    indice += 1
print(lista_precios)
    

['Desde 54.668 €', 'Desde 54.668 €', 'Desde 38.280 €', 'Desde 38.280 €', 'Desde 51.478 €', 'Desde 51.478 €', 'Desde 40.001 €', 'Desde 40.001 €', 'Desde 34.885 €', 'Desde 34.885 €', 'Desde 42.981 €', 'Desde 42.981 €', 'Desde 45.963 €', 'Desde 45.963 €', 'Desde 32.660 €', 'Desde 32.660 €', 'Desde 46.330 €', 'Desde 46.330 €', 'Desde 59.790 €', 'Desde 59.790 €']


In [23]:
# Para obtener una lista con sólo uno de los valores
lista_precios_corregida = [lista_precios[i] for i in range(1, len(lista_precios), 2)]
print(lista_precios_corregida)

['Desde 54.668 €', 'Desde 38.280 €', 'Desde 51.478 €', 'Desde 40.001 €', 'Desde 34.885 €', 'Desde 42.981 €', 'Desde 45.963 €', 'Desde 32.660 €', 'Desde 46.330 €', 'Desde 59.790 €']


In [24]:
# Añadimos a la lista del DF
lista_precios_DF.extend(lista_precios_corregida)
lista_precios_DF

['Desde 54.668 €',
 'Desde 38.280 €',
 'Desde 51.478 €',
 'Desde 40.001 €',
 'Desde 34.885 €',
 'Desde 42.981 €',
 'Desde 45.963 €',
 'Desde 32.660 €',
 'Desde 46.330 €',
 'Desde 59.790 €']

In [25]:
# Última columna
valoraciones= sopa.find_all(class_='wowscore-pill')
lista_valoraciones=[]
for indice, valoracion in enumerate(valoraciones):
    lista_valoraciones.append(valoracion.text.strip())
    indice +=1
print(lista_valoraciones)

['9/10', '9/10', '8/10', '8/10', '8/10', '8/10', '7/10', '9/10', '8/10', '8/10']


In [26]:
# Añadimos a la lista del DF
lista_valoraciones_DF.extend(lista_valoraciones)

In [27]:
# Creo un diccionario, para generar mi primer dataframe, al que luego iremos añadiendo más datos
diccionario_coches = {
    'Nombre': lista_coches_DF,
    'Valoraciones': lista_valoraciones_DF,
    'A favor': lista_a_favor_DF,
    'En contra': lista_en_contra_DF,
    'Opiniones': lista_opinion_DF,
    'Precio': lista_precios_DF
   
}

print(diccionario_coches)

{'Nombre': ['1. Mazda CX-80', '2. Renault Espace', '3. Hyundai Santa Fe', '4. Skoda Kodiaq', '5. Peugeot 5008', '6. Volkswagen Tayron', '7. KIA Sorento', '8. Nissan X-Trail', '9. Mercedes-Benz GLB', '10. Toyota Highlander'], 'Valoraciones': ['9/10', '9/10', '8/10', '8/10', '8/10', '8/10', '7/10', '9/10', '8/10', '8/10'], 'A favor': [['Excelente eficiencia tanto en el PHEV como en el diésel', 'Fácil acceso y amplio espacio en la segunda y tercera fila de asientos', 'Alta calidad de construcción y acabados interiores'], ['Consumo', 'Equipamiento disponible', 'Opción de siete plazas sin coste'], ['Equipamiento y tecnología propios de coches de más alta gama', 'Motor híbrido potente y tracción total', 'Es un coche cómodo de conducir en toda circunstancia'], ['Practicidad y espacio disponible', 'Oferta mecánica amplia', 'Confort de marcha'], ['Amplio interior con 7 plazas de serie', 'Comodidad y refinamiento', 'Motorizaciones híbridas y eléctricas, con buena autonomía'], ['Maletero muy gran

In [28]:
# Genero el DataFrame
df = pd.DataFrame.from_dict(diccionario_coches)
df

,Nombre,Valoraciones,A favor,En contra,Opiniones,Precio
0,1. Mazda CX-80,9/10,[Excelente eficiencia tanto en el PHEV como en...,[La suspensión puede sentirse algo seca en cie...,El CX-80 usa la misma plataforma y motores que...,Desde 54.668 €
1,2. Renault Espace,9/10,"[Consumo, Equipamiento disponible, Opción de s...","[Funcionamiento poco refinado en ocasiones, Ga...",El nuevo Renault Espace es ahora un SUV con un...,Desde 38.280 €
2,3. Hyundai Santa Fe,8/10,[Equipamiento y tecnología propios de coches d...,"[Precio elevado, Escasa autonomía eléctrica de...",El Santa Fe dispone de motorizaciones híbrida ...,Desde 51.478 €
3,4. Skoda Kodiaq,8/10,"[Practicidad y espacio disponible, Oferta mecá...","[Ergonomía de algunos mandos, Plazas de la ter...","El Skoda Kodiaq es mucho más barato que el Q7,...",Desde 40.001 €
4,5. Peugeot 5008,8/10,"[Amplio interior con 7 plazas de serie, Comodi...",[La calidad del sistema multimedia podría mejo...,El Peugeot 5008 es un coche muy redondo y cohe...,Desde 34.885 €
5,6. Volkswagen Tayron,8/10,"[Maletero muy grande, Aislamiento, Integración...","[PHEV solo de cinco plazas, La tercera fila de...",El Volkswagen Tayron es un SUV espacioso y tec...,Desde 42.981 €
6,7. KIA Sorento,7/10,"[Variedad de motores, Amplio y disponible con ...","[Consumo, Mandos del climatizador, Prestacione...",El KIA Sorento es un SUV de gran tamaño que pr...,Desde 45.963 €
7,8. Nissan X-Trail,9/10,"[Presentación vistosa, Confort general, Uno de...","[Rumorosidad en ciertas condiciones, Presencia...",El X-Trail ha subido sensiblemente de nivel en...,Desde 32.660 €
8,9. Mercedes-Benz GLB,8/10,"[Versatilidad, Diseño, Variedad]","[Precio, Equipamiento en modelos básicos, Sin ...",Este SUV de carrocería compacta por ofrecer ca...,Desde 46.330 €
9,10. Toyota Highlander,8/10,"[Versatilidad, Facilidad de conducción, Etique...","[Poca variedad mecánica, Tamaño, Manejo multim...",El Highlander tiene un sistema de propulsión h...,Desde 59.790 €


In [29]:
# Segunda url
url_2='https://www.autohero.com/es/consejos/elegir/mejor-coche/suv-7-plazas/'

In [30]:
# Voy a crear una función para automatizar la conexion y la comprobación del estado de la misma
# La sopa la generaré en las propias funciones que me van a servir para 'automatizar' el proceso de web scraping 
def accesibilidad (url):
    conexion = requests.get(url)
    print(conexion.status_code, conexion.reason)


In [31]:
# Comprobando conexión
accesibilidad(url_2)

200 OK


In [32]:
# Función para extaer los nombres de los coches 
def lista_coches (url, tag, class_):
    '''
    Debemos obtener los tag y atributos que necesitamos de la página web (y se pasa en modo str)

    '''
    conexion = requests.get(url)
    sopa = bs(conexion.text, "lxml")
    coches = sopa.find_all( tag, class_) # no consigo que me coja la tupla con los tags
    lista_coches = []
    for indice, coche in enumerate(coches):
        lista_coches.append(coche.text.strip())
        indice += 1
    return lista_coches

In [33]:
# Obtengo el listado de coches y luego lo añado a la lista del DF
listado_coches = lista_coches(url_2, 'h3', class_="title___HASgI title___2DV4T underline___dbvD3 variant-h2___1eoCd color-secondary___3vq1b weight-normal___25n2H align-center___17w2J")

In [34]:
lista_coches_DF.extend(listado_coches)
lista_coches_DF

['1. Mazda CX-80',
 '2. Renault Espace',
 '3. Hyundai Santa Fe',
 '4. Skoda Kodiaq',
 '5. Peugeot 5008',
 '6. Volkswagen Tayron',
 '7. KIA Sorento',
 '8. Nissan X-Trail',
 '9. Mercedes-Benz GLB',
 '10. Toyota Highlander',
 'Volvo XC90',
 'Audi Q7',
 'Mercedes-Benz Clase GLB',
 'Hyundai Santa Fe',
 'Kia Sorento',
 'Peugeot 5008',
 'SEAT Tarraco',
 'Skoda Kodiaq',
 'Nissan X-Trail',
 'Mitsubishi Outlander']

In [35]:
# Valoraciones , esta página no tiene valoraciones como tal, asi que se cumplimentan con None
lista_valoraciones_url_2 = crear_lista_none(10)
# Añadimos a la del DF
lista_valoraciones_DF.extend(lista_valoraciones_url_2)
lista_valoraciones_DF

['9/10',
 '9/10',
 '8/10',
 '8/10',
 '8/10',
 '8/10',
 '7/10',
 '9/10',
 '8/10',
 '8/10',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [36]:
# En esta página viene una descripción con características que me parecen interesantes.
# Como en la página anterior no existía este apartado, lo primero que voy a hacer es generar la lista pertinente para el Df
# A continuación genero una lista con valores nulos para los vehiculos de la primera url y los añado a la lista DF
lista_caracteristicas_DF = []
lista_caracteristicas_url_1 = crear_lista_none(10)
lista_caracteristicas_DF.extend(lista_caracteristicas_url_1)

In [37]:
# Obtencion de las características
conexion = requests.get(url_2)
sopa = bs(conexion.text, "lxml")
caracteristicas = sopa.find_all('div', class_="text___3mq1J")
lista = []
for índice, caracteristica in enumerate(caracteristicas):
    lista.append(caracteristica.text.strip())
    indice += 1
print(lista) # Obtengo todo el texto que hay en la página, tengo que dejar en la lista aquello que me interesa

['La fiebre de los SUV sigue creciendo en España, y tanto los SUV pequeños como los de mayor tamaño han ganado cuota de mercado en los últimos años. En esta guía, seleccionamos los mejores SUV con 7 plazas, destacando ventajas como la amplitud del habitáculo y la gran capacidad de carga del maletero.', 'Si formas parte de una familia numerosa, es fundamental encontrar un vehículo amplio y modular que asegure la comodidad de todos los pasajeros. Tanto para los desplazamientos diarios como para viajes largos, opta por un coche con un gran volumen de maletero y una excelente habitabilidad. Los coches con 7 asientos, especialmente los SUV, son ideales, ofreciendo espacio y flexibilidad.Actualmente, los SUV han reemplazado al monovolumen como la opción preferida para las familias grandes. Estos SUV no solo brindan más espacio, sino también una mejor capacidad off-road y mayor versatilidad para distintos tipos de terrenos y condiciones climáticas.Al seleccionar un SUV de siete plazas, es imp

In [38]:
# Tengo que borrar las 2 primeras entradas de la lista y de la 13 en adelante.
del lista[0:2] 
del lista[10:]
lista

['El Volvo XC90 es un SUV de 7 plazas que destaca por su espacioso interior y diseño ergonómico. Con una longitud de 4,95 metros, ofrece un maletero de 692 litros con cinco asientos en uso y 314 litros con las siete plazas ocupadas. Los asientos son ajustables individualmente, optimizando el confort para todos los ocupantes, incluso en la tercera fila, donde pueden viajar adultos de hasta 170 cm.El interior está acabado con materiales de alta calidad como cuero y madera, y el puesto de conducción está diseñado para una experiencia intuitiva. Además, el XC90 viene equipado de serie con tecnologías como control de crucero adaptativo y faros LED. Sin embargo, la suspensión puede resultar firme a bajas velocidades y el sistema de infoentretenimiento depende en gran medida de la pantalla táctil.Ventajas del Volvo XC90Materiales interiores premium.Confort excepcional en todos los asientos.Equipamiento tecnológico avanzado.Desventajas del Volvo XC90Suspensión firme a bajas velocidades.Sistema

In [39]:
# La añado a la lista DF
lista_caracteristicas_DF.extend(lista)

In [40]:
# Creo una función para cosas a favor
def lo_mejor (url, tag, class_):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    lista_a_favor = []
    cosas_a_favor= sopa.find_all(tag, class_) 
    for indice, a_favor in enumerate(cosas_a_favor):
        lista_a_favor.append(a_favor.text.strip()) 
    return lista_a_favor

In [41]:
# Creo una función para las cosas en contra
def lo_peor (url, tag, class_):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    lista_en_contra = []
    cosas_en_contra= sopa.find_all(tag, class_) 
    for indice, en_contra in enumerate(cosas_en_contra):
        lista_en_contra.append(en_contra.text.strip()) 
    return lista_en_contra

In [42]:
lista_a_favor_url_2 = lo_mejor(url_2, 'ul', None)
lista_a_favor_url_2

['Materiales interiores premium.Confort excepcional en todos los asientos.Equipamiento tecnológico avanzado.',
 'Suspensión firme a bajas velocidades.Sistema de infoentretenimiento dependiente de la pantalla táctil.',
 'Motores potentes y variados.Tracción quattro y suspensión adaptativa.Interior lujoso y tecnología avanzada.',
 'Consumo de combustible elevado.Precio inicial alto.Tamaño que dificulta la maniobrabilidad en ciudad.',
 'Interior modular con asientos ajustables.Tecnología avanzada (MBUX).Calidad de los acabados superior.',
 'Precio de opciones elevado.Tercera fila adecuada solo para pasajeros pequeños.Acceso complicado a la tercera fila.',
 'Diseño moderno y asientos cómodos.Pantalla táctil grande y clara.Sistemas de seguridad avanzados.',
 'Tercera fila ajustada para adultos altos.Algunos materiales interiores menos lujosos.Consumo de combustible elevado en versiones potentes.',
 'Diseño moderno y flexible.Pantallas avanzadas y claras.Conducción cómoda y segura.',
 'Terce

In [43]:
# Veo que me da tanto a favor como en contra y dos registros no válidos
# Primero elimino los dos ultimos registros
del lista_a_favor_url_2[-2 :]
lista_a_favor_url_2


['Materiales interiores premium.Confort excepcional en todos los asientos.Equipamiento tecnológico avanzado.',
 'Suspensión firme a bajas velocidades.Sistema de infoentretenimiento dependiente de la pantalla táctil.',
 'Motores potentes y variados.Tracción quattro y suspensión adaptativa.Interior lujoso y tecnología avanzada.',
 'Consumo de combustible elevado.Precio inicial alto.Tamaño que dificulta la maniobrabilidad en ciudad.',
 'Interior modular con asientos ajustables.Tecnología avanzada (MBUX).Calidad de los acabados superior.',
 'Precio de opciones elevado.Tercera fila adecuada solo para pasajeros pequeños.Acceso complicado a la tercera fila.',
 'Diseño moderno y asientos cómodos.Pantalla táctil grande y clara.Sistemas de seguridad avanzados.',
 'Tercera fila ajustada para adultos altos.Algunos materiales interiores menos lujosos.Consumo de combustible elevado en versiones potentes.',
 'Diseño moderno y flexible.Pantallas avanzadas y claras.Conducción cómoda y segura.',
 'Terce

In [44]:
# Ahora nos quedamos con los valores impares para lo puntos positivos
lista_a_favor_url_2_corregida =[lista_a_favor_url_2[i] for i in range(0, len(lista_a_favor_url_2), 2)]
lista_a_favor_url_2_corregida

['Materiales interiores premium.Confort excepcional en todos los asientos.Equipamiento tecnológico avanzado.',
 'Motores potentes y variados.Tracción quattro y suspensión adaptativa.Interior lujoso y tecnología avanzada.',
 'Interior modular con asientos ajustables.Tecnología avanzada (MBUX).Calidad de los acabados superior.',
 'Diseño moderno y asientos cómodos.Pantalla táctil grande y clara.Sistemas de seguridad avanzados.',
 'Diseño moderno y flexible.Pantallas avanzadas y claras.Conducción cómoda y segura.',
 'Diseño moderno y atractivo.Pantalla panorámica grande y avanzada.Segunda fila de asientos ajustables individualmente.',
 'Espacio interior amplio y bien acabado.Sistemas de asistencia avanzados.Buena conectividad (Apple CarPlay, Android Auto).',
 'Interior bien diseñado y espacioso.Equipamiento tecnológico avanzado.Buenas opciones de motorización.',
 'Sistema e-Power eficiente.Interior espacioso y versátil.Buen equipamiento de seguridad.',
 'Espacio interior amplio y versátil

In [45]:
# Misma operacion con los valores pares para obtener los puntos negativos
lista_en_contra_url_2 =[lista_a_favor_url_2[i] for i in range(1, len(lista_a_favor_url_2), 2)]
lista_en_contra_url_2

['Suspensión firme a bajas velocidades.Sistema de infoentretenimiento dependiente de la pantalla táctil.',
 'Consumo de combustible elevado.Precio inicial alto.Tamaño que dificulta la maniobrabilidad en ciudad.',
 'Precio de opciones elevado.Tercera fila adecuada solo para pasajeros pequeños.Acceso complicado a la tercera fila.',
 'Tercera fila ajustada para adultos altos.Algunos materiales interiores menos lujosos.Consumo de combustible elevado en versiones potentes.',
 'Tercera fila ajustada para adultos.Espacio de carga reducido con tercera fila en uso.Pérdida de potencia en altitudes elevadas.',
 'Tercera fila ajustada para adultos altos.Espacio de carga limitado con siete plazas en uso.Algunos materiales interiores menos lujosos.',
 'No es el más deportivo de la gama.Falta de una versión TGI de Gas Natural Comprimido.Precio inicial elevado.',
 'Espacio ajustado para cinco adultos.Chasis menos equilibrado que en competidores.Ruido perceptible del motor diésel.',
 'Consumo de combus

In [46]:
# Las añadimos a sus listas del DF
lista_a_favor_DF.extend(lista_a_favor_url_2_corregida)
lista_en_contra_DF.extend(lista_en_contra_url_2)

In [47]:
# Columna opiniones
# Ahora hago una función para sacar el texto que argumenta las opiniones
def opiniones (url, tag, class_):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    lista_opiniones = []
    opiniciones= sopa.find_all(tag, class_)
    for indice, opinion in enumerate(opiniciones):
        lista_opiniones.append(opinion.text.strip())
        indice += 1
    return lista_opiniones

In [48]:
lista_opiniones_url_2 = opiniones(url_2, 'p', None)
lista_opiniones_url_2

['La fiebre de los SUV sigue creciendo en España, y tanto los SUV pequeños como los de mayor tamaño han ganado cuota de mercado en los últimos años. En esta guía, seleccionamos los mejores SUV con 7 plazas, destacando ventajas como la amplitud del habitáculo y la gran capacidad de carga del maletero.',
 'Si formas parte de una familia numerosa, es fundamental encontrar un vehículo amplio y modular que asegure la comodidad de todos los pasajeros. Tanto para los desplazamientos diarios como para viajes largos, opta por un coche con un gran volumen de maletero y una excelente habitabilidad. Los coches con 7 asientos, especialmente los SUV, son ideales, ofreciendo espacio y flexibilidad.',
 'Actualmente, los SUV han reemplazado al monovolumen como la opción preferida para las familias grandes. Estos SUV no solo brindan más espacio, sino también una mejor capacidad off-road y mayor versatilidad para distintos tipos de terrenos y condiciones climáticas.',
 'Al seleccionar un SUV de siete pla

In [49]:
# Me devuelve todos los párrafos, toca volver a escoger los que me interesan
# Me quedo con: 11, 16, 21, 26, 31, 36, 41, 46, 51, 56 (ojo, son las posiciones reales, no las de la lista Python)
del lista_opiniones_url_2 [0:10]
del lista_opiniones_url_2 [-8:] # elimino las últimas
lista_opiniones_url_2_corregida = lista_opiniones_url_2 [::5] # me quedo con el resto que son las que ocupan las posiciones 5

In [50]:
lista_opiniones_url_2_corregida

['Este SUV de origen sueco combina lujo, espacio y versatilidad. Su interior de alta calidad, junto con su capacidad para siete ocupantes y un generoso maletero, lo hacen ideal para familias que buscan confort y practicidad. Además, su equipamiento de seguridad y tecnología lo posiciona como una excelente opción en su segmento.',
 'Recomendamos el Audi Q7 por su combinación de lujo, espacio y tecnología avanzada. Es ideal para familias que buscan confort y prestaciones sin comprometer el estilo. Su versatilidad, desde la ciudad hasta el todoterreno, lo convierte en una opción atractiva para quienes desean un SUV premium de gran tamaño.',
 'Recomendamos el GLB por su versatilidad y practicidad excepcionales. Combina el lujo de Mercedes con un espacio interior impresionante y la opción de siete plazas, algo poco común en su segmento. Su diseño inteligente y tecnología avanzada lo convierten en una excelente opción para familias que buscan un SUV premium y funcional.',
 'Recomendamos el H

In [51]:
# La añadimos a la columna del DF
lista_opinion_DF.extend(lista_opiniones_url_2_corregida)

In [52]:
# En esta página no vienen precios, por lo que genero una lista con None y la añado a la del DF
lista_precios_url_2 = crear_lista_none(10)
lista_precios_DF.extend(lista_precios_url_2)  
lista_precios_DF   

['Desde 54.668 €',
 'Desde 38.280 €',
 'Desde 51.478 €',
 'Desde 40.001 €',
 'Desde 34.885 €',
 'Desde 42.981 €',
 'Desde 45.963 €',
 'Desde 32.660 €',
 'Desde 46.330 €',
 'Desde 59.790 €',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [53]:
# Tengo que añadir la nueva lista de caracteristicas al diccionario para ampliar mi DF
diccionario_coches['Características']= lista_caracteristicas_DF


In [54]:
# Genero el DF actualizado
df = pd.DataFrame.from_dict(diccionario_coches)
df

,Nombre,Valoraciones,A favor,En contra,Opiniones,Precio,Características
0,1. Mazda CX-80,9/10,[Excelente eficiencia tanto en el PHEV como en...,[La suspensión puede sentirse algo seca en cie...,El CX-80 usa la misma plataforma y motores que...,Desde 54.668 €,None
1,2. Renault Espace,9/10,"[Consumo, Equipamiento disponible, Opción de s...","[Funcionamiento poco refinado en ocasiones, Ga...",El nuevo Renault Espace es ahora un SUV con un...,Desde 38.280 €,None
2,3. Hyundai Santa Fe,8/10,[Equipamiento y tecnología propios de coches d...,"[Precio elevado, Escasa autonomía eléctrica de...",El Santa Fe dispone de motorizaciones híbrida ...,Desde 51.478 €,None
3,4. Skoda Kodiaq,8/10,"[Practicidad y espacio disponible, Oferta mecá...","[Ergonomía de algunos mandos, Plazas de la ter...","El Skoda Kodiaq es mucho más barato que el Q7,...",Desde 40.001 €,None
4,5. Peugeot 5008,8/10,"[Amplio interior con 7 plazas de serie, Comodi...",[La calidad del sistema multimedia podría mejo...,El Peugeot 5008 es un coche muy redondo y cohe...,Desde 34.885 €,None
5,6. Volkswagen Tayron,8/10,"[Maletero muy grande, Aislamiento, Integración...","[PHEV solo de cinco plazas, La tercera fila de...",El Volkswagen Tayron es un SUV espacioso y tec...,Desde 42.981 €,None
6,7. KIA Sorento,7/10,"[Variedad de motores, Amplio y disponible con ...","[Consumo, Mandos del climatizador, Prestacione...",El KIA Sorento es un SUV de gran tamaño que pr...,Desde 45.963 €,None
7,8. Nissan X-Trail,9/10,"[Presentación vistosa, Confort general, Uno de...","[Rumorosidad en ciertas condiciones, Presencia...",El X-Trail ha subido sensiblemente de nivel en...,Desde 32.660 €,None
8,9. Mercedes-Benz GLB,8/10,"[Versatilidad, Diseño, Variedad]","[Precio, Equipamiento en modelos básicos, Sin ...",Este SUV de carrocería compacta por ofrecer ca...,Desde 46.330 €,None
9,10. Toyota Highlander,8/10,"[Versatilidad, Facilidad de conducción, Etique...","[Poca variedad mecánica, Tamaño, Manejo multim...",El Highlander tiene un sistema de propulsión h...,Desde 59.790 €,None


In [55]:
# Tercera página
url_3='http://quecochemecompro.com/blog/comparativa-suv-7-plazas-guia-de-compra/'

In [56]:
accesibilidad(url_3)

200 OK


In [57]:
# Extraemos el listado de coches
lista_coches_url_3 = lista_coches( url_3,'h3', class_="font-bold text-28 leading-none")
lista_coches_url_3 

['1. Mazda CX-80',
 '2. Skoda Kodiaq',
 '3. Kia Sorento',
 '4. Hyundai Santa Fe Híbrido',
 '5. Volkswagen Tayron híbrido',
 '6. Toyota Highlander Híbrido',
 '7. Peugeot 5008',
 '8. Volkswagen Tiguan Allspace',
 '9. Mercedes GLB',
 '10. Mercedes EQB eléctrico',
 '1. Range Rover',
 '2. Volvo XC90',
 '3. Volvo EX90 eléctrico',
 '4. Kia EV9',
 '5. Audi Q7']

In [58]:
# Sólo me salen 15 de los 20 que vienen
# Conseguí sacar los otros 5
listado_coches_url_3_faltantes = lista_coches(url_3,'h3', class_="mb-3 font-bold text-28" )
listado_coches_url_3_faltantes

['6. BMW X5',
 '7. Mercedes GLE',
 '8. Mercedes GLS',
 '9. BMW X7',
 '10. Hongqi eHS9']

In [59]:
# Añado ambas listas a la del DF
lista_coches_DF.extend(lista_coches_url_3)
lista_coches_DF.extend(listado_coches_url_3_faltantes)
lista_coches_DF

['1. Mazda CX-80',
 '2. Renault Espace',
 '3. Hyundai Santa Fe',
 '4. Skoda Kodiaq',
 '5. Peugeot 5008',
 '6. Volkswagen Tayron',
 '7. KIA Sorento',
 '8. Nissan X-Trail',
 '9. Mercedes-Benz GLB',
 '10. Toyota Highlander',
 'Volvo XC90',
 'Audi Q7',
 'Mercedes-Benz Clase GLB',
 'Hyundai Santa Fe',
 'Kia Sorento',
 'Peugeot 5008',
 'SEAT Tarraco',
 'Skoda Kodiaq',
 'Nissan X-Trail',
 'Mitsubishi Outlander',
 '1. Mazda CX-80',
 '2. Skoda Kodiaq',
 '3. Kia Sorento',
 '4. Hyundai Santa Fe Híbrido',
 '5. Volkswagen Tayron híbrido',
 '6. Toyota Highlander Híbrido',
 '7. Peugeot 5008',
 '8. Volkswagen Tiguan Allspace',
 '9. Mercedes GLB',
 '10. Mercedes EQB eléctrico',
 '1. Range Rover',
 '2. Volvo XC90',
 '3. Volvo EX90 eléctrico',
 '4. Kia EV9',
 '5. Audi Q7',
 '6. BMW X5',
 '7. Mercedes GLE',
 '8. Mercedes GLS',
 '9. BMW X7',
 '10. Hongqi eHS9']

In [60]:
# Valoraciones
# Funcion para obtener la valoración
def valoraciones(url, tag, class_):
    # defino dos variables (tag y atributo) por si tengo que poner más de uno. A veces solo necesito poner un tag, por lo que a atributo le doy por defecto None, para que no me de problemas)
    conexion = requests.get(url)
    sopa= bs(conexion.text, "lxml")
    lista_valoraciones = []
    valoraciones= sopa.find_all(tag, class_)
    for indice, valoracion in enumerate(valoraciones):
        lista_valoraciones.append(valoracion.text.strip())
        indice +=1
    return lista_valoraciones

In [61]:
lista_valoraciones_url_3 = valoraciones(url_3, 'div', class_="inline-block bg-wine/[.1] text-20 rounded px-1") # 15 primeras valoraciones

In [62]:
# Sólo obtengo 15 y por más que busco es como si faltara parte de código en la web
# como sólo son 5 las voy a añadir manualmente
# Creo una lista con las valoraciones que faltan y se la añado a la que va a formar el DF
lista_valoraciones_url_3_manual = [ '7,8/10', '7,8/10', '7,1/10', '6,9/10', '6,8/10']
 

In [63]:
# Las añado a las del DF
lista_valoraciones_DF.extend(lista_valoraciones_url_3)
lista_valoraciones_DF.extend(lista_valoraciones_url_3_manual)

In [64]:
# Puntos a favor
lista_a_favor_url_3 = lo_mejor (url_3, 'div', class_="text-gray-dark text-16 sm:text-17 leading-5") 

In [65]:
lista_a_favor_url_3

['Hasta siete plazas, presentación, opción de tracción total',
 'Gama inicial limitada, precio en acabados superiores',
 'Espacio, maletero, siete plazas, calidad de terminación',
 'Dimensiones, consumos algo elevados, precio',
 'Arsenal tecnológico, confort en marcha, mejora de acabados',
 'Precio elevado para lo que busca un cliente en Hyundai',
 'Imagen, equipamiento, espacio interior, mecánica muy fiable y resolutiva',
 'Caja de cambios, consumos a altas velocidades',
 'Diseño interior, calidad de construcción, espacio interior',
 'Precio, gama mecánica limitada',
 'Calidad de rodadura, gama de motores',
 'Diseño muy convencional, detalles de acabado',
 'Equipamiento disponible, hasta siete asientos, comportamiento',
 'Precio elevado, opciones caras, maletero con siete asientos en uso justo',
 'Capacidad para 7 ocupantes, prestaciones, autonomía',
 'Precio elevado, algunas calidades mejorables',
 'Diseño, interior, calidad de acabados, capacidades todoterreno en otra liga, suspensi

In [66]:
# Saca tanto lo bueno como lo malo porque tienen el mismo tag y atributo. Debo hacer alguna modificación para escoger el impar en las de a favor y la par en las de en contra
lista_lo_mejor_url_3 = [lista_a_favor_url_3[i] for i in range(0, len(lista_a_favor_url_3), 2)] # impares
lista_lo_peor_url_3 = [lista_a_favor_url_3[i] for i in range(1, len(lista_a_favor_url_3), 2)] # pares

In [67]:
lista_lo_mejor_url_3

['Hasta siete plazas, presentación, opción de tracción total',
 'Espacio, maletero, siete plazas, calidad de terminación',
 'Arsenal tecnológico, confort en marcha, mejora de acabados',
 'Imagen, equipamiento, espacio interior, mecánica muy fiable y resolutiva',
 'Diseño interior, calidad de construcción, espacio interior',
 'Calidad de rodadura, gama de motores',
 'Equipamiento disponible, hasta siete asientos, comportamiento',
 'Capacidad para 7 ocupantes, prestaciones, autonomía',
 'Diseño, interior, calidad de acabados, capacidades todoterreno en otra liga, suspensión neumática',
 'Diseño interior y exterior, calidad de construcción, seguridad, espacio interior para 7 adultos, confort',
 'Calidad de acabados, hasta siete plazas, equipo, autonomía',
 'Espacio interior, calidad de construcción, tecnología, diseño interior']

In [68]:
# Compruebo que hay vehículos que no tienen dichas valoraciones, asi que añado los valores None
introducir_nulos(lista_lo_mejor_url_3, (0,4,13))
introducir_nulos(lista_lo_peor_url_3, (0,4,13))

[None, 'Hasta siete plazas, presentación, opción de tracción total', 'Espacio, maletero, siete plazas, calidad de terminación', 'Arsenal tecnológico, confort en marcha, mejora de acabados', None, 'Imagen, equipamiento, espacio interior, mecánica muy fiable y resolutiva', 'Diseño interior, calidad de construcción, espacio interior', 'Calidad de rodadura, gama de motores', 'Equipamiento disponible, hasta siete asientos, comportamiento', 'Capacidad para 7 ocupantes, prestaciones, autonomía', 'Diseño, interior, calidad de acabados, capacidades todoterreno en otra liga, suspensión neumática', 'Diseño interior y exterior, calidad de construcción, seguridad, espacio interior para 7 adultos, confort', 'Calidad de acabados, hasta siete plazas, equipo, autonomía', None, 'Espacio interior, calidad de construcción, tecnología, diseño interior']
[None, 'Gama inicial limitada, precio en acabados superiores', 'Dimensiones, consumos algo elevados, precio', 'Precio elevado para lo que busca un cliente 

In [69]:
# Añadimos a las listas del DF
lista_a_favor_DF.extend(lista_lo_mejor_url_3)
lista_en_contra_DF.extend(lista_lo_peor_url_3)

In [70]:
# Me faltan los 5 datos en ambas listas que no he logrado sacar, por lo que los introduzco a mano al ser sólo 5
lista_lo_mejor_url_3_manual = ['Tacto de conducción', 'Diseño, calidades, posibilidad de 7 plazas, capacidad todoterreno', 'Calidad de rodadura, confort, cargatecnológica', None, None]
lista_lo_peor_url_3_manual = ['Precio elevado, opciones caras', 'Precio elevado, opciones caras', 'Dimensiones, precio de algunos opcionales, ciertos ajustes', None, None]

In [71]:
# Añadimos a las listas del DF
lista_a_favor_DF.extend(lista_lo_mejor_url_3_manual)
lista_en_contra_DF.extend(lista_lo_peor_url_3_manual)

In [72]:
# Columna de opiniones
# Ahora hago una función para sacar el texto que argumenta las opiniones
def opiniones (url, tag, class_):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    lista_opiniones = []
    opiniciones= sopa.find_all(tag, class_)
    for indice, opinion in enumerate(opiniciones):
        lista_opiniones.append(opinion.text.strip())
        indice += 1
    return lista_opiniones

In [73]:
# El tag es 'p', asi que me va a dar muchos párrafos
lista_opiniones_url_3 = opiniones(url_3, 'p', None)

In [74]:
lista_opiniones_url_3

['Os presentamos la guía definitiva de los mejores SUV de 7 plazas por su relación calidad-precio. En ella podréis encontrar todos los modelos a la venta que cumplen estas dos características: una carrocería SUV familiar y 7 plazas extra disponibles. Si no necesitas las 7 plazas, echa un vistazo a nuestra guía de los mejores SUV grandes.',
 'Para una visión global de todos los modelos disponibles, recomendamos nuestra guía de compra de coches de 7 plazas, mientras que para orientarnos a todoterrenos realmente duros para uso fuera del asfalto, contamos con la guía de auténticos todoterrenos. También contamos con una guía con todos los coches de 7 plazas del mercado, que incluye todo tipo de carrocerías. Si lo más importante para ti es el precio, puedes echar un vistazo a nuestra guía con todos los SUV baratos.',
 'Los SUV compactos son una de las opciones preferidas por los compradores, al condensar, en un único vehículo, el máximo número de cualidades posibles. Se trata de coches altos

In [75]:
# Creo una funcion para obtener todos los párrafos que me interesan de una lista y agruparlos en una nueva lista 
def agregar_un_parrafo(lista, posicion):
    lista_parrafos= []
    parrafo = lista[posicion]
    lista_parrafos.append(parrafo)
    return lista_parrafos

In [76]:
# Comienzo a agregar párrafos a mi lista
# Listado párrafos = 8, 12, 17, 22, 28, 32, 37, 42, 47, 52, 59, 64, 69, 74, 79// a partir de aqui son los 5 que no consigo 82, 84, 86, 90, 90 (estos dos últimos no tienen reseña) 
opiniones_corregidas = []
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 8))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 12))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 17))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 22))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 28)) # 5
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 32))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 37))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 42))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 47))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 52)) # 10
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 59))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 64))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 69))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 74))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 79)) #15
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 82))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 84))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 86))
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 90)) # No tiene en la web 91 son string vacios
opiniones_corregidas.append(agregar_un_parrafo(lista_opiniones_url_3, 90)) # No tiene en la web 91 son string vacios

In [77]:
opiniones_corregidas

[['Para que puedas valorar mejor las diferencias y las similitudes entre Mazda CX-80 y Mercedes GLC, tienes a tu disposición nuestra comparativa destacada donde podrás hacerlo de manera rápida y sencilla. El cliente que quiere comprar un vehículo de este segmento encontrará en el Mercedes GLC una alternativa al Mazda CX-80, al que consideramos su rival más claro.'],
 ['Sin duda, el gran SUV checo es una de las mejores opciones de su segmento, destacando por una relación calidad-precio casi imbatible, mucho espacio interior, un rodar muy suave y silencioso y unos acabados que nada tienen que envidiar a otros vehículos de la competencia.'],
 ['Con un diseño mucho más elaborado y un interior que lo sitúa casi en la línea de las marcas premium, el Kia Sorento se convierte así en una excelente opción para los que busquen un SUV de gran tamaño, con buenas dosis de espacio y no quieran abonar lo que las firmas premium están pidiendo por lo que sería el modelo equivalente.'],
 ['El Hyundai San

In [78]:
# Ahora toca añadirlo a la lista del DF
lista_opinion_DF.extend(opiniones_corregidas)

In [79]:
# Columna de precios
# Creo una función para sacar precios, como en la anterior, primero obtengo el listado de precios y luego lo uno a la del DF
def lista_precios (url, tag, class_):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    precios= sopa.find_all(tag, class_)
    lista_precios = []
    for indice, precio in enumerate(precios):
        lista_precios.append(precio.text.strip())
        indice += 1
    return lista_precios
    

In [80]:
# Listado de precios
lista_precios_url_3 = lista_precios(url_3, 'span', class_='block text-primary text-20 leading-tight') # obtengo 15 

In [81]:
# Añado los 5 restantes a mano
lista_precios_url_3_manual = ['94.037€', '91.914€', '118.594€', '120.510€', '93.989€']

In [82]:
# Agregamos a la lista del Df
lista_precios_DF.extend(lista_precios_url_3)
lista_precios_DF.extend(lista_precios_url_3_manual)

In [83]:
# Finalmente obtenemos la lista de caracteristicas, que no tiene
# Genero una lista con 20 valores None
lista_caracteristicas_url_3 = crear_lista_none(20)
lista_caracteristicas_DF.extend(lista_caracteristicas_url_3)

In [84]:
len(lista_coches_DF)
len(lista_valoraciones_DF)
len(lista_a_favor_DF)
len(lista_en_contra_DF)
len(lista_opinion_DF)
len(lista_precios_DF)
len(lista_caracteristicas_DF)

40

In [85]:
df = pd.DataFrame.from_dict(diccionario_coches)
df

,Nombre,Valoraciones,A favor,En contra,Opiniones,Precio,Características
0,1. Mazda CX-80,9/10,[Excelente eficiencia tanto en el PHEV como en...,[La suspensión puede sentirse algo seca en cie...,El CX-80 usa la misma plataforma y motores que...,Desde 54.668 €,None
1,2. Renault Espace,9/10,"[Consumo, Equipamiento disponible, Opción de s...","[Funcionamiento poco refinado en ocasiones, Ga...",El nuevo Renault Espace es ahora un SUV con un...,Desde 38.280 €,None
2,3. Hyundai Santa Fe,8/10,[Equipamiento y tecnología propios de coches d...,"[Precio elevado, Escasa autonomía eléctrica de...",El Santa Fe dispone de motorizaciones híbrida ...,Desde 51.478 €,None
3,4. Skoda Kodiaq,8/10,"[Practicidad y espacio disponible, Oferta mecá...","[Ergonomía de algunos mandos, Plazas de la ter...","El Skoda Kodiaq es mucho más barato que el Q7,...",Desde 40.001 €,None
4,5. Peugeot 5008,8/10,"[Amplio interior con 7 plazas de serie, Comodi...",[La calidad del sistema multimedia podría mejo...,El Peugeot 5008 es un coche muy redondo y cohe...,Desde 34.885 €,None
5,6. Volkswagen Tayron,8/10,"[Maletero muy grande, Aislamiento, Integración...","[PHEV solo de cinco plazas, La tercera fila de...",El Volkswagen Tayron es un SUV espacioso y tec...,Desde 42.981 €,None
6,7. KIA Sorento,7/10,"[Variedad de motores, Amplio y disponible con ...","[Consumo, Mandos del climatizador, Prestacione...",El KIA Sorento es un SUV de gran tamaño que pr...,Desde 45.963 €,None
7,8. Nissan X-Trail,9/10,"[Presentación vistosa, Confort general, Uno de...","[Rumorosidad en ciertas condiciones, Presencia...",El X-Trail ha subido sensiblemente de nivel en...,Desde 32.660 €,None
8,9. Mercedes-Benz GLB,8/10,"[Versatilidad, Diseño, Variedad]","[Precio, Equipamiento en modelos básicos, Sin ...",Este SUV de carrocería compacta por ofrecer ca...,Desde 46.330 €,None
9,10. Toyota Highlander,8/10,"[Versatilidad, Facilidad de conducción, Etique...","[Poca variedad mecánica, Tamaño, Manejo multim...",El Highlander tiene un sistema de propulsión h...,Desde 59.790 €,None


In [86]:
# Cuarta url
url_4 = 'https://www.diariomotor.com/guias-compra/mejores-suv-7-plazas/'
accesibilidad(url_4)

200 OK


In [87]:
# Lista de coches (extraigo e inserto a la lista del DF)
lista_coches_url_4 = lista_coches(url_4, 'h2', class_="m-0 text-3xl leading-none -tracking-xs md:-tracking-s md:text-4xl")
lista_coches_DF.extend(lista_coches_url_4)
len(lista_coches_DF) # para comprobar si se han añadido todos los datos

67

In [88]:
lista_coches_url_4

['Skoda Kodiaq',
 'Peugeot 5008 híbrido',
 'Volkswagen Tayron híbrido',
 'Peugeot 5008 eléctrico',
 'Peugeot 5008',
 'Volkswagen Tiguan Allspace',
 'Nissan X-Trail híbrido',
 'Ebro s800 híbrido',
 'Nissan X-Trail',
 'Renault Espace',
 'Kia Sorento',
 'Skoda Kodiaq híbrido',
 'Volkswagen Tayron híbrido',
 'Volkswagen Tayron',
 'Mercedes GLB',
 'Renault Espace',
 'KGM (SsangYong) Rexton',
 'Range Rover',
 'Volvo XC90',
 'Volvo EX90 eléctrico',
 'Kia EV9',
 'Mazda CX-80 híbrido',
 'Mazda CX-80',
 'Audi Q7',
 'Volvo XC90 Híbrido',
 'Kia Sorento',
 'BMW X5']

In [89]:
# Lista Valoraciones (extraigo e inserto en la lista del DF)
lista_valoraciones_url_4 = valoraciones(url_4, 'div', class_="mt-1 leading-7 numero")
lista_valoraciones_DF.extend(lista_valoraciones_url_4)
len(lista_valoraciones_DF)

67

In [90]:
# Lista a favor (no hay datos en esta página)
# Genero una lista con valores nulos
lista_a_favor_url_4 = crear_lista_none(27)
lista_a_favor_DF.extend(lista_a_favor_url_4)
len(lista_a_favor_DF)

67

In [91]:
# Lista en contra (no hay datos en esta página)
# Genero una lista con valores nulos
lista_en_contra_url_4 = crear_lista_none(27)
lista_en_contra_DF.extend(lista_en_contra_url_4)
len(lista_en_contra_DF)

67

In [92]:
# Opiniones (extraigo datos y agrego a la lista del DF)
lista_opiniones_url_4 = opiniones(url_4, 'div',  class_="py-4 border-b border-gray-100 cuerpo")
lista_opinion_DF.extend(lista_opiniones_url_4)
len(lista_opinion_DF)

67

In [93]:
# Precios (extraigo datos y agrego a la lista del DF)
lista_precios_url_4 = lista_precios(url_4, 'div', class_="flex flex-col text-3xl font-semibold leading-7 text-main md:text-4xl -tracking-xs md:mt-0 md:mr-2")
lista_precios_DF.extend(lista_precios_url_4)
len(lista_precios_DF)

67

In [94]:
# Características ( en la página vienen cuatro datos que voy a incluir aquí: etiqueta, potencia, consumo, maletero)
# Voy a crear las listas por separado, y luego las uno en una sola para añadirla a características
# Creo función para las características
def caracteristicas (url, tag, class_):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    caracteristicas= sopa.find_all(tag, class_)
    lista_caracteristicas = []
    for indice, caracteristica in enumerate(caracteristicas):
        lista_caracteristicas.append(caracteristica.text.strip())
        indice += 1
    return lista_caracteristicas

In [95]:
# Función para extraer las etiquetas
def lista_etiquetas (url):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    etiquetado = sopa.find_all("img", {"alt": True})
# Creo la lista de las etiquetas
    nombre_pegatinas = []
    for nombre in etiquetado:
        pegatina = nombre["alt"] # recojo los nombres de las pegatinas con la primera letra de cada palabra en mayúscula ( para normaizar ya datos)
        nombre_pegatinas.append(pegatina)
        etiqueta = [x for x in nombre_pegatinas if "Etiqueta ambiental " in x] # de la lista me quedo con las que se llaman Etiqueta medioambiental
        lista_etiquetas = []
        lista_etiquetas.append(etiqueta)
    return lista_etiquetas


In [96]:
lista_etiquetas_url_4 = lista_etiquetas(url_4)
lista_etiquetas_url_4 # Me devuelve una lista de listas

[['Etiqueta ambiental C',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental C',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental C',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental C',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental C',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental C',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental C',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental C',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental C',
  'Etiqueta ambiental ECO',
  'Etiqueta ambiental zero',
  'Etiqueta ambiental C',
  'Etiqueta ambiental ECO']]

In [97]:
lista_etiquetas_url_4_corregida = lista_etiquetas_url_4[0] # Me quedo con el listado de etiquetas

In [98]:
# Agrupar
# Función para dos párrafos
def agregar_dos_parrafos (lista, posicion_1, posicion_2):
    lista_agrupada =[]
    parrafo_A = lista[posicion_1] 
    parrafo_B = lista[posicion_2]
    parrafo = [parrafo_A + parrafo_B]
    lista_agrupada.append(parrafo)
    return lista_agrupada

# Agrupar
# Función para 3 párrafos
def agregar_tres_parrafos (lista, posicion_1, posicion_2, posicion_3):
    lista_agrupada = []
    parrafo_A = lista[posicion_1] 
    parrafo_B = lista[posicion_2]
    parrafo_C = lista[posicion_3]
    parrafo = [parrafo_A + parrafo_B + parrafo_C]
    lista_agrupada.append(parrafo)
    return lista_agrupada

In [99]:
lista_etiquetas_corregida =[]
lista_etiquetas_corregida.append(agregar_dos_parrafos(lista_etiquetas_url_4_corregida, 0, 1))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 2))
lista_etiquetas_corregida.append(agregar_tres_parrafos(lista_etiquetas_url_4_corregida, 3, 4, 5))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 6))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 7))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 8))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 9))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 10))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 11))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 12)) # 10 vehiculos
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 13))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 14))
lista_etiquetas_corregida.append(agregar_tres_parrafos(lista_etiquetas_url_4_corregida, 15, 16, 17))
lista_etiquetas_corregida.append(agregar_tres_parrafos(lista_etiquetas_url_4_corregida, 18, 19, 20))
lista_etiquetas_corregida.append(agregar_dos_parrafos(lista_etiquetas_url_4_corregida, 21, 22))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 23))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 24))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 25))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 26))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 27))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 28))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 29))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 30))
lista_etiquetas_corregida.append(agregar_dos_parrafos(lista_etiquetas_url_4_corregida, 31, 32))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 33))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 34))
lista_etiquetas_corregida.append(agregar_un_parrafo(lista_etiquetas_url_4_corregida, 35))

In [100]:
lista_etiquetas_corregida

[[['Etiqueta ambiental CEtiqueta ambiental ECO']],
 ['Etiqueta ambiental zero'],
 [['Etiqueta ambiental ECOEtiqueta ambiental CEtiqueta ambiental zero']],
 ['Etiqueta ambiental zero'],
 ['Etiqueta ambiental ECO'],
 ['Etiqueta ambiental C'],
 ['Etiqueta ambiental ECO'],
 ['Etiqueta ambiental zero'],
 ['Etiqueta ambiental ECO'],
 ['Etiqueta ambiental ECO'],
 ['Etiqueta ambiental C'],
 ['Etiqueta ambiental zero'],
 [['Etiqueta ambiental ECOEtiqueta ambiental CEtiqueta ambiental zero']],
 [['Etiqueta ambiental ECOEtiqueta ambiental CEtiqueta ambiental zero']],
 [['Etiqueta ambiental ECOEtiqueta ambiental C']],
 ['Etiqueta ambiental ECO'],
 ['Etiqueta ambiental C'],
 ['Etiqueta ambiental ECO'],
 ['Etiqueta ambiental ECO'],
 ['Etiqueta ambiental zero'],
 ['Etiqueta ambiental zero'],
 ['Etiqueta ambiental zero'],
 ['Etiqueta ambiental ECO'],
 [['Etiqueta ambiental CEtiqueta ambiental ECO']],
 ['Etiqueta ambiental zero'],
 ['Etiqueta ambiental C'],
 ['Etiqueta ambiental ECO']]

In [101]:
len(lista_etiquetas_corregida) # Compruebo si coincide con el número de vehículos de la página

27

In [102]:
# Lista, potencia
lista_potencia = caracteristicas (url_4, 'div', class_="flex flex-col text-base font-semibold leading-4 text-gray-700 uppercase texto -tracking-xs")
len(lista_potencia)
lista_potencia

['Potencia 150-265 CV',
 'Consumo 5.30-8.20 l/100Km',
 'Maletero 910',
 'Potencia 195 CV',
 'Consumo 1.00-1.00 l/100Km',
 'Maletero 348',
 'Potencia 150-272 CV',
 'Autonomia 734.177 - 11250 KM',
 'Recarga 0.43 h',
 'Maletero 345',
 'Potencia 214-231 CV',
 'Autonomia 500 - 668 KM',
 'Recarga 0.45 - 0.5 h',
 'Maletero 348',
 'Potencia 145 CV',
 'Consumo 5.80-5.80 l/100Km',
 'Maletero 348',
 'Potencia 150-190 CV',
 'Consumo 5.70-8.50 l/100Km',
 'Maletero 700',
 'Potencia 204-213 CV',
 'Consumo 5.70-6.40 l/100Km',
 'Maletero 575',
 'Potencia 347 CV',
 'Autonomia 789.474 KM',
 'Recarga 0.33 h',
 'Maletero',
 'Potencia 163 CV',
 'Consumo 6.90-7.20 l/100Km',
 'Maletero 585',
 'Potencia 200 CV',
 'Consumo 4.60-5.00 l/100Km',
 'Maletero 159',
 'Potencia 194 CV',
 'Consumo 6.20-6.80 l/100Km',
 'Maletero 187',
 'Potencia 204 CV',
 'Consumo 0.40-0.50 l/100Km',
 'Maletero 910',
 'Potencia 150-272 CV',
 'Autonomia 734.177 - 11250 KM',
 'Recarga 0.43 h',
 'Maletero 345',
 'Potencia 150-272 CV',
 'Aut

In [103]:
introducir_nulos(lista_potencia, (3,7,19,23,27,35,39,43,47,59,63,67,71,75,87,91,95,99,103,107))

['Potencia 150-265 CV', 'Consumo 5.30-8.20 l/100Km', 'Maletero 910', None, 'Potencia 195 CV', 'Consumo 1.00-1.00 l/100Km', 'Maletero 348', None, 'Potencia 150-272 CV', 'Autonomia 734.177 - 11250 KM', 'Recarga 0.43 h', 'Maletero 345', 'Potencia 214-231 CV', 'Autonomia 500 - 668 KM', 'Recarga 0.45 - 0.5 h', 'Maletero 348', 'Potencia 145 CV', 'Consumo 5.80-5.80 l/100Km', 'Maletero 348', None, 'Potencia 150-190 CV', 'Consumo 5.70-8.50 l/100Km', 'Maletero 700', None, 'Potencia 204-213 CV', 'Consumo 5.70-6.40 l/100Km', 'Maletero 575', None, 'Potencia 347 CV', 'Autonomia 789.474 KM', 'Recarga 0.33 h', 'Maletero', 'Potencia 163 CV', 'Consumo 6.90-7.20 l/100Km', 'Maletero 585', None, 'Potencia 200 CV', 'Consumo 4.60-5.00 l/100Km', 'Maletero 159', None, 'Potencia 194 CV', 'Consumo 6.20-6.80 l/100Km', 'Maletero 187', None, 'Potencia 204 CV', 'Consumo 0.40-0.50 l/100Km', 'Maletero 910', None, 'Potencia 150-272 CV', 'Autonomia 734.177 - 11250 KM', 'Recarga 0.43 h', 'Maletero 345', 'Potencia 150-272

In [104]:
# Genero una función para poder agrupar los valores de 4 en 4
def agrupar_de_cuatro(lista): 
  '''
  Agrupa los elementos de una lista en sublistas de tamaño 4

  Args:
  lista: La lista original.

  Returns:
  Una nueva lista que contiene sublistas de tamaño 4.
  '''
  return [lista[i:i+4] for i in range(0, len(lista), 4)]

In [105]:
lista_agrupada_caracteristicas = agrupar_de_cuatro(lista_potencia)
len(lista_agrupada_caracteristicas)

27

In [106]:
lista_agrupada_caracteristicas

[['Potencia 150-265 CV', 'Consumo 5.30-8.20 l/100Km', 'Maletero 910', None],
 ['Potencia 195 CV', 'Consumo 1.00-1.00 l/100Km', 'Maletero 348', None],
 ['Potencia 150-272 CV',
  'Autonomia 734.177 - 11250 KM',
  'Recarga 0.43 h',
  'Maletero 345'],
 ['Potencia 214-231 CV',
  'Autonomia 500 - 668 KM',
  'Recarga 0.45 - 0.5 h',
  'Maletero 348'],
 ['Potencia 145 CV', 'Consumo 5.80-5.80 l/100Km', 'Maletero 348', None],
 ['Potencia 150-190 CV', 'Consumo 5.70-8.50 l/100Km', 'Maletero 700', None],
 ['Potencia 204-213 CV', 'Consumo 5.70-6.40 l/100Km', 'Maletero 575', None],
 ['Potencia 347 CV', 'Autonomia 789.474 KM', 'Recarga 0.33 h', 'Maletero'],
 ['Potencia 163 CV', 'Consumo 6.90-7.20 l/100Km', 'Maletero 585', None],
 ['Potencia 200 CV', 'Consumo 4.60-5.00 l/100Km', 'Maletero 159', None],
 ['Potencia 194 CV', 'Consumo 6.20-6.80 l/100Km', 'Maletero 187', None],
 ['Potencia 204 CV', 'Consumo 0.40-0.50 l/100Km', 'Maletero 910', None],
 ['Potencia 150-272 CV',
  'Autonomia 734.177 - 11250 KM',


In [107]:
# A continuación debo unir las dos listas, pero que se combinen, para que generen una sola con 27 elementos
lista_caracteristicas_url_4 = []
for indice, item in enumerate(lista_agrupada_caracteristicas):
    lista_caracteristicas_url_4.append(item+lista_etiquetas_corregida[indice])
print(lista_caracteristicas_url_4)

[['Potencia 150-265 CV', 'Consumo 5.30-8.20 l/100Km', 'Maletero 910', None, ['Etiqueta ambiental CEtiqueta ambiental ECO']], ['Potencia 195 CV', 'Consumo 1.00-1.00 l/100Km', 'Maletero 348', None, 'Etiqueta ambiental zero'], ['Potencia 150-272 CV', 'Autonomia 734.177 - 11250 KM', 'Recarga 0.43 h', 'Maletero 345', ['Etiqueta ambiental ECOEtiqueta ambiental CEtiqueta ambiental zero']], ['Potencia 214-231 CV', 'Autonomia 500 - 668 KM', 'Recarga 0.45 - 0.5 h', 'Maletero 348', 'Etiqueta ambiental zero'], ['Potencia 145 CV', 'Consumo 5.80-5.80 l/100Km', 'Maletero 348', None, 'Etiqueta ambiental ECO'], ['Potencia 150-190 CV', 'Consumo 5.70-8.50 l/100Km', 'Maletero 700', None, 'Etiqueta ambiental C'], ['Potencia 204-213 CV', 'Consumo 5.70-6.40 l/100Km', 'Maletero 575', None, 'Etiqueta ambiental ECO'], ['Potencia 347 CV', 'Autonomia 789.474 KM', 'Recarga 0.33 h', 'Maletero', 'Etiqueta ambiental zero'], ['Potencia 163 CV', 'Consumo 6.90-7.20 l/100Km', 'Maletero 585', None, 'Etiqueta ambiental ECO

In [108]:
len(lista_caracteristicas_url_4)

27

In [109]:
# Añadimos la lista a la del DF
lista_caracteristicas_DF.extend(lista_caracteristicas_url_4)

In [110]:
df = pd.DataFrame.from_dict(diccionario_coches)
df

,Nombre,Valoraciones,A favor,En contra,Opiniones,Precio,Características
0,1. Mazda CX-80,9/10,[Excelente eficiencia tanto en el PHEV como en...,[La suspensión puede sentirse algo seca en cie...,El CX-80 usa la misma plataforma y motores que...,Desde 54.668 €,None
1,2. Renault Espace,9/10,"[Consumo, Equipamiento disponible, Opción de s...","[Funcionamiento poco refinado en ocasiones, Ga...",El nuevo Renault Espace es ahora un SUV con un...,Desde 38.280 €,None
2,3. Hyundai Santa Fe,8/10,[Equipamiento y tecnología propios de coches d...,"[Precio elevado, Escasa autonomía eléctrica de...",El Santa Fe dispone de motorizaciones híbrida ...,Desde 51.478 €,None
3,4. Skoda Kodiaq,8/10,"[Practicidad y espacio disponible, Oferta mecá...","[Ergonomía de algunos mandos, Plazas de la ter...","El Skoda Kodiaq es mucho más barato que el Q7,...",Desde 40.001 €,None
4,5. Peugeot 5008,8/10,"[Amplio interior con 7 plazas de serie, Comodi...",[La calidad del sistema multimedia podría mejo...,El Peugeot 5008 es un coche muy redondo y cohe...,Desde 34.885 €,None
...,...,...,...,...,...,...,...
62,Mazda CX-80,8.3,None,None,,56.198 € Ahorra 5.023 €,"[Potencia 254 CV, Consumo 5.80-5.80 l/100Km, M..."
63,Audi Q7,8.1,None,None,El Audi Q7 ya es un viejo conocido en el secto...,74.460 € Ahorra 10.283 €,"[Potencia 231-507 CV, Consumo 7.80-12.20 l/100..."
64,Volvo XC90 Híbrido,8.0,None,None,El Volvo XC90 híbrido es un SUV de siete plaza...,80.350 € Ahorra 8.413 €,"[Potencia 455 CV, Consumo 1.20-1.30 l/100Km, M..."
65,Kia Sorento,7.8,None,None,El kia Sorento es una de las opciones más sens...,49.880 € Ahorra 2.972 €,"[Potencia 194 CV, Consumo 6.20-6.80 l/100Km, M..."


In [111]:
# Quinta url, introduzco también monovolumenes
url_5 = 'https://www.quecochemecompro.com/blog/coches-de-7-plazas-todos-los-modelos/'
# Conexiones
accesibilidad(url_5)

200 OK


In [112]:
# Lista coches (extraer y añadir a las listas DF)
lista_coches_url_5 = lista_coches(url_5, 'h3', class_="font-bold text-28 leading-none")
len(lista_coches_url_5)

15

In [113]:
# Como la vez anterior sólo obtengo 15 resultados, habiendo en este caso 30 en la web
lista_coches_url_5_B = lista_coches(url_5, 'h3', class_="mb-3 font-bold text-28")
len(lista_coches_url_5_B)

15

In [114]:
# Los añado a la lista del DF
lista_coches_DF.extend(lista_coches_url_5)
lista_coches_DF.extend(lista_coches_url_5_B)
len(lista_coches_DF)

97

In [115]:
# Valoraciones (extraer y añadir a las listas DF)(puede que tenga el mismo problema que antes)
lista_valoraciones_url_5 = valoraciones(url_5, 'div', class_="inline-block bg-wine/[.1] text-20 rounded px-1")
lista_valoraciones_url_5_B = ('4,9/10', '5,4/10', '5,6/10', '7,3/10', '6,5/10', '7,9/10', '6,8/10', '7,3/10', '6,5/10', '7,8/10', '7,8/10', '8,6/10', '8,1/10', '7,8/10', '7,8/10')
lista_valoraciones_DF.extend(lista_valoraciones_url_5)
lista_valoraciones_DF.extend(lista_valoraciones_url_5_B)
len(lista_valoraciones_DF)

97

In [116]:
# Lista a favor (extraer y añadir a las listas del DF)(mismo procedimiento url_3)
lista_a_favor_url_5 = lo_mejor (url_5, 'div', class_="text-gray-dark text-16 sm:text-17 leading-5") 
len(lista_a_favor_url_5)

22

In [117]:
lista_a_favor_url_5

['Amplitud, maletero, hasta siete plazas',
 'Acabados, dotación justa, sonoridad elevada',
 'Equipamiento disponible, ventanillas traseras eléctricas, motores eficientes, presentación, luneta practicable, espacio interior, versatilidad',
 'Portón trasero muy grande, sonoridad en los diésel',
 'Sencillez y practicidad también para familias',
 'Calidad de los revestimientos',
 'Calidad de rodadura, diseño, posibilidades de equipamiento',
 'Precio, refinamiento del motor diésel',
 'Posibilidad de 7 plazas, espacio interior, volumen de maletero, calidad de construcción, versatilidad interior',
 'Estética cuadrada, precio elevado, diseño sobrio',
 'Comportamiento, gama variada, equipo disponible',
 'Detalles de acabado, ventanillas traseras con apertura "de compás"',
 'Diseño, capacidad para 7 ocupantes, carga tecnológica, posibilidad de camperización',
 'Materiales no acordes con el precio, el Renault Kangoo es el mismo coche pero más barato',
 'Siete plazas, equipamiento disponible, preci

In [118]:
# Hay 4 de los 15 primeros coches que no tienen ni a favor, ni en contra (6,8,10,11)
# Añado nulos en esas posiciones
introducir_nulos(lista_a_favor_url_5, (10,11, 14,15, 18,19, 20,21))
lista_a_favor_url_5

['Amplitud, maletero, hasta siete plazas', 'Acabados, dotación justa, sonoridad elevada', 'Equipamiento disponible, ventanillas traseras eléctricas, motores eficientes, presentación, luneta practicable, espacio interior, versatilidad', 'Portón trasero muy grande, sonoridad en los diésel', 'Sencillez y practicidad también para familias', 'Calidad de los revestimientos', 'Calidad de rodadura, diseño, posibilidades de equipamiento', 'Precio, refinamiento del motor diésel', 'Posibilidad de 7 plazas, espacio interior, volumen de maletero, calidad de construcción, versatilidad interior', 'Estética cuadrada, precio elevado, diseño sobrio', None, None, 'Comportamiento, gama variada, equipo disponible', 'Detalles de acabado, ventanillas traseras con apertura "de compás"', None, None, 'Diseño, capacidad para 7 ocupantes, carga tecnológica, posibilidad de camperización', 'Materiales no acordes con el precio, el Renault Kangoo es el mismo coche pero más barato', None, None, None, None, 'Siete plaz

['Amplitud, maletero, hasta siete plazas',
 'Acabados, dotación justa, sonoridad elevada',
 'Equipamiento disponible, ventanillas traseras eléctricas, motores eficientes, presentación, luneta practicable, espacio interior, versatilidad',
 'Portón trasero muy grande, sonoridad en los diésel',
 'Sencillez y practicidad también para familias',
 'Calidad de los revestimientos',
 'Calidad de rodadura, diseño, posibilidades de equipamiento',
 'Precio, refinamiento del motor diésel',
 'Posibilidad de 7 plazas, espacio interior, volumen de maletero, calidad de construcción, versatilidad interior',
 'Estética cuadrada, precio elevado, diseño sobrio',
 None,
 None,
 'Comportamiento, gama variada, equipo disponible',
 'Detalles de acabado, ventanillas traseras con apertura "de compás"',
 None,
 None,
 'Diseño, capacidad para 7 ocupantes, carga tecnológica, posibilidad de camperización',
 'Materiales no acordes con el precio, el Renault Kangoo es el mismo coche pero más barato',
 None,
 None,
 Non

In [119]:
len(lista_a_favor_url_5)

30

In [120]:
# Los 5 restantes hasta 20, no tienen tampoco ni en contra ni a favor
# Añado nulos en esas posiciones
introducir_nulos(lista_a_favor_url_5, (30,31,32,33,34,35,36,37,38,39))
len(lista_a_favor_url_5)                

['Amplitud, maletero, hasta siete plazas', 'Acabados, dotación justa, sonoridad elevada', 'Equipamiento disponible, ventanillas traseras eléctricas, motores eficientes, presentación, luneta practicable, espacio interior, versatilidad', 'Portón trasero muy grande, sonoridad en los diésel', 'Sencillez y practicidad también para familias', 'Calidad de los revestimientos', 'Calidad de rodadura, diseño, posibilidades de equipamiento', 'Precio, refinamiento del motor diésel', 'Posibilidad de 7 plazas, espacio interior, volumen de maletero, calidad de construcción, versatilidad interior', 'Estética cuadrada, precio elevado, diseño sobrio', None, None, 'Comportamiento, gama variada, equipo disponible', 'Detalles de acabado, ventanillas traseras con apertura "de compás"', None, None, 'Diseño, capacidad para 7 ocupantes, carga tecnológica, posibilidad de camperización', 'Materiales no acordes con el precio, el Renault Kangoo es el mismo coche pero más barato', None, None, None, None, 'Siete plaz

40

In [121]:
# Intento sacar los 10 vehiculos que me quedan, pero finalmente los añado manualmente
lista_a_favor_url_5_B = ['Hasta siete plazas, presentación, opción de tracción total', 'Gama inicial limitada, precio en acabados superiores', 'Amplitud interior, disponibilidad de 7 plazas, confort de marcha, equipamiento muy completo', 'Precios algo elevados, pocas opciones de personalizacion', 'Diseño interior, calidad de construccion, espacio interior', 'Precio, gama mecánica limitada', 'Amplitud, maletero con cinco asientos, diseño, presentación', 'Aspecto de algunos plásticos, dimensiones, motores pequeños algo justos', 'Diseño, calidades, posibilidad 7 plazas, capacidad todoterreno', 'Precio elevado, opciones caras', 'Tacto de conducción, tecnología, calidades, diseño', 'Precio elevado, opciones caras', 'Diseño interior y exterior, calidad de construcción, seguridad, espacio interior para 7 adultos, confort', 'Agilidad limitada para su tamaño, precio de opciones', 'Espacio ingerior, calidad de construcción, tecnología, diseño interior', 'Proporciones exteriores algo raras, precio', 'Espacio, maletero, 7 plazas, calidad de terminación', 'Dimensiones. consumos algo elevados, precio', 'Arsenal tecnológico, confort en marcha, mejora de acabados', 'Precio elevado para lo que busca un cliente Hyundai' ]

In [122]:
# Uno ambas listas (de me quedará un listado de 60 frases que corresponden tanto a lo mejor como lo mejorable)
lista_a_favor_url_5.extend(lista_a_favor_url_5_B)
len(lista_a_favor_url_5)

60

In [123]:
# Tengo que sacar las dos listas : a favor y en contra
lista_lo_mejor_url_5 = [lista_a_favor_url_5[i] for i in range(0, len(lista_a_favor_url_5), 2)] # impares
lista_lo_peor_url_5 = [lista_a_favor_url_5[i] for i in range(1, len(lista_a_favor_url_5), 2)] # pares

In [124]:
# Las añado a la lista del DF
lista_a_favor_DF.extend(lista_lo_mejor_url_5)
lista_en_contra_DF.extend(lista_lo_mejor_url_5)

In [125]:
len(lista_a_favor_DF)

97

In [126]:
len(lista_en_contra_DF)

97

In [127]:
# Opiniones (extraer y añadir a la lista de DF)
lista_opiniones_url_5 = opiniones(url_5, 'p', None)


In [128]:
lista_opiniones_url_5

['Los coches de 7 plazas han visto incrementarse su demanda en los últimos tiempos, impulsados por el auge del segmento monovolumen en un primer momento y más tarde de la mano de los formatos SUV. El coche familiar de 7 plazas tiene algo mágico en su promesa de transportar a familia y amigos en un momento puntual o a familias numerosas en el día a día.',
 '',
 'Podemos dividir a los coches de 7 plazas en los dos formatos en los que pueden presentarse: monovolumen y SUV, pero esta es una guía en la que vamos a recorrer el mercado completo, de menor a mayor precio, para que cada uno elija el coche que más le convenza en el formato que más le guste.',
 'Un monovolumen es, por definición, el vehículo familiar por excelencia. Pese a que en los últimos años su presencia se ha visto claramente desplazada por la explosión del fenómeno SUV, su practicidad y utilidad cuando se trata de transportar grandes cargas o un buen número de personas siguen siendo, sin ninguna duda, imbatibles. Los monovo

In [129]:
# Creo la lista agregando los párrafos que me interesan
lista_opiniones_url_5_corregida = []
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 8))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 13))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 18))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 23))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 28))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 34))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 38))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 44))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 48))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 54))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 62))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 66))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 71))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 76))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 81))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 84))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 85))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 86))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 87))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 88))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 96))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 98))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 100))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 102)) # Espace
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 104))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 106))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 108))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 110))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 112))
lista_opiniones_url_5_corregida.append(agregar_un_parrafo (lista_opiniones_url_5, 114))


In [130]:
lista_opiniones_url_5_corregida

[['El modelo más asequible para viajar en familia de la firma rumana, es también más interesante que el anterior Lodgy. El nuevo Jogger aterriza con cinco o siete plazas, motores eficientes y la deseada etiqueta ambiental ECO de la DGT.'],
 ['El diseño de la Peugeot Rifter es el de una furgoneta mucho más refinada que lo que habíamos visto hasta la fecha, combinado a la perfección con un habitáculo más cuidado que nunca. La furgoneta compacta del fabricante francés se convierte así en una gran opción para las familias más polivalentes.'],
 ['Para los que buscan un Toyota monovolumen, esta es la respuesta; un vehículo industrial. Sí, Toyota se beneficia de las sinergías empresariales y lanza su primer vehículo fabricado en España, una variante prima-hermana de las Citroën Berlingo y Peugeot Rifter'],
 ['Con un aspecto futurista y una construcción directamente derivada de modelos como el Volkswagen Golf, si alguien está buscando un comercial compacto con una tecnología de primera, en la 

In [131]:
# Añado a la lista DF
lista_opinion_DF.extend(lista_opiniones_url_5_corregida)

In [132]:
len(lista_opinion_DF)

97

In [133]:
# Precios (extraer datos y añadirlos a la lista DF)
lista_precios_url_5 = lista_precios(url_5, 'span', class_="block text-primary text-20 leading-tight")
len(lista_precios_url_5)

15

In [134]:
# Añado manualmentelos otros 15
lista_precios_url_5_manual =['15.990€', '25.999€', '29.900€', '37.660€', '21.400€', '37.550€', '32.900€', '35.010€', '37.830€', '91.914€', '94.037€', '71.900€', '74.460€', '49.710€', '51.210€']

In [135]:
# Añadimos los datos a la lista del DF
lista_precios_DF.extend(lista_precios_url_5)
lista_precios_DF.extend(lista_precios_url_5_manual)
len(lista_precios_DF)

97

In [136]:
# Carácterísticas (en esta página no hay info de este tipo)
# Genero una lista de 30 None
lista_caracteristicas_url_5 = crear_lista_none(30)
lista_caracteristicas_DF.extend(lista_caracteristicas_url_5)
len(lista_caracteristicas_DF)

97

In [137]:
# Nos pasamos a las monovolumen
url_6 = 'https://www.autohero.com/es/consejos/elegir/mejor-coche/monovolumen-7-plazas/'
accesibilidad(url_6)

200 OK


In [138]:
# Lista de vehiculos (extraer y pasar datos a las listas del DF)
lista_coches_url_6 = lista_coches(url_6, 'h3', class_="title___HASgI title___2DV4T underline___dbvD3 variant-h2___1eoCd color-secondary___3vq1b weight-normal___25n2H align-center___17w2J")
lista_coches_DF.extend(lista_coches_url_6)
len(lista_coches_DF)

107

In [139]:
# Lista valoraciones(no tiene)
# Generamos lista con nulos
lista_valoraciones_url_6 = crear_lista_none(10)
lista_valoraciones_DF.extend(lista_valoraciones_url_6)
len(lista_valoraciones_DF)

107

In [140]:
# Lista a favor (extraer y pasar datos a las listas del DF)
lista_a_favor_url_6 = lo_mejor (url_6, 'p', None)
lista_a_favor_url_6
len(lista_a_favor_url_6)

78

In [141]:
lista_a_favor_url_6

['Última actualización: 02.04.2025',
 '¿Estás pensando en comprar un monovolumen de 7 plazas? Puede ser una decisión muy acertada si necesitas más espacio, comodidad y versatilidad en tu día a día. Estos vehículos están diseñados para adaptarse a las necesidades de familias numerosas o personas que viajan con frecuencia en grupo, ofreciendo una solución práctica y funcional.',
 'Los monovolúmenes de 7 plazas destacan por su capacidad de adaptarse a las necesidades de familias numerosas o quienes viajan en grupo con frecuencia. Su diseño ofrece un equilibrio perfecto entre espacio, comodidad y funcionalidad, siendo una alternativa práctica frente a otros tipos de vehículos como los SUV grandes.',
 'Algunas de las ventajas principales de estos coches incluyen:',
 'Los monovolúmenes familiares, como la Volkswagen Transporter y la Mercedes Clase V, destacan por su alta retención de valor en el mercado de segunda mano, gracias a su popularidad y versatilidad.Lutz Grunwald - Experto en calid

In [142]:
# En el listado están tanto las cosas a favor como en contra y algunas que no me interesan
lista_favor = []
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 8))
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 8))
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 19))
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 26))
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 33))
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 40))
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 47))
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 54))
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 61))
lista_favor.append(agregar_un_parrafo(lista_a_favor_url_6, 68))

lista_contra = []
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 9))
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 9))
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 20))
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 27))
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 34))
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 41))
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 48))
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 55))
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 62))
lista_contra.append(agregar_un_parrafo(lista_a_favor_url_6, 69)) 
lista_a_favor_DF.extend(lista_favor)
lista_en_contra_DF.extend(lista_contra)
len(lista_en_contra_DF)

107

In [143]:
# Lista opiniones (extraer y añadir los datos a la lista del DF)
lista_opiniones_url_6 = opiniones(url_6, 'p', None)
lista_opiniones_url_6

['Última actualización: 02.04.2025',
 '¿Estás pensando en comprar un monovolumen de 7 plazas? Puede ser una decisión muy acertada si necesitas más espacio, comodidad y versatilidad en tu día a día. Estos vehículos están diseñados para adaptarse a las necesidades de familias numerosas o personas que viajan con frecuencia en grupo, ofreciendo una solución práctica y funcional.',
 'Los monovolúmenes de 7 plazas destacan por su capacidad de adaptarse a las necesidades de familias numerosas o quienes viajan en grupo con frecuencia. Su diseño ofrece un equilibrio perfecto entre espacio, comodidad y funcionalidad, siendo una alternativa práctica frente a otros tipos de vehículos como los SUV grandes.',
 'Algunas de las ventajas principales de estos coches incluyen:',
 'Los monovolúmenes familiares, como la Volkswagen Transporter y la Mercedes Clase V, destacan por su alta retención de valor en el mercado de segunda mano, gracias a su popularidad y versatilidad.Lutz Grunwald - Experto en calid

In [144]:
# Salen todos los párrafos. Me quedo con: 8,12,16,20,24,28,32,36,40,44(posiciones python)
lista_opiniones_url_6_corregida = []
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 7))
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 10))
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 18))
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 25))
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 32))
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 39))
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 46))
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 53))
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 60))
lista_opiniones_url_6_corregida.append(agregar_un_parrafo(lista_opiniones_url_6, 67))
lista_opiniones_url_6_corregida

[['El maletero ofrece 610 litros con 7 plazas y hasta 1.410 litros si lo configuras con 6. Además, puedes elegir entre versiones de 6, 7 u 8 plazas según lo que necesites en cada momento.'],
 ['Porque es ideal si priorizas el lujo, el confort y la versatilidad para viajar en familia o en grupo, sin renunciar a la calidad y la tecnología de Mercedes.'],
 ['El maletero tiene 189 litros de capacidad con las 7 plazas ocupadas y llega hasta 533 litros si solo usas 5 asientos. Además, gracias al sistema One Touch, puedes plegar los asientos de forma rápida y sencilla con solo pulsar un botón, adaptando el espacio según tus necesidades.'],
 ['El maletero varía desde 207 litros con las 7 plazas en uso hasta 2.617 litros si abates todos los asientos, lo que te da mucha flexibilidad para adaptarlo a tus necesidades.'],
 ['El maletero varía entre 267 litros con las 7 plazas en uso y hasta 2.430 litros si abates los asientos, lo que te permite adaptar el espacio según tus necesidades.'],
 ['El mal

In [145]:
# Lo añadimos a la lista del DF
lista_opinion_DF.extend(lista_opiniones_url_6_corregida)
len(lista_opinion_DF)

107

In [146]:
# Lista Precios (no hay en la web)
lista_precios_url_6 = crear_lista_none(10)
lista_precios_DF.extend(lista_precios_url_6)
len(lista_precios_DF)

107

In [147]:
# Lista características (extraer y añadir los datos a la lista del DF)
lista_caracteristicas_url_6 = caracteristicas(url_6, 'p', None)
# Me tengo que quedar con los párrafos que  me interesan : 6-7, 10-11, 14-15, 18-19, 22-23, 26-27, 30-31, 34-35, 38-39, 42-43
lista_caracteristicas_url_6_corregida = []
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 6, 7))
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 10, 11))
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 14, 15))
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 18, 19))
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 22, 23))
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 26, 27))
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 30, 31))
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 34, 35))
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 38, 39))
lista_caracteristicas_url_6_corregida.append(agregar_dos_parrafos(lista_opiniones_url_6, 42, 43))
lista_caracteristicas_url_6_corregida

[[['Si buscas espacio y lujo en un vehículo, el Mercedes-Benz Clase V es difícil de superar. Este monovolumen no solo es amplio, sino que también lleva el confort a otro nivel. Con una longitud de entre 4,90 y 5,37 metros, es perfecto para familias grandes o para quienes quieren viajar con estilo y comodidad.El maletero ofrece 610 litros con 7 plazas y hasta 1.410 litros si lo configuras con 6. Además, puedes elegir entre versiones de 6, 7 u 8 plazas según lo que necesites en cada momento.']],
 [['Porque es ideal si priorizas el lujo, el confort y la versatilidad para viajar en familia o en grupo, sin renunciar a la calidad y la tecnología de Mercedes.Viajar en grupo nunca fue tan cómodo: el Mercedes-Benz Clase V combina lujo, tecnología y amplitud en cada trayecto.']],
 [['El maletero tiene una capacidad de 610 litros con 7 plazas y hasta 1.410 litros configurado con 6. Además, puedes elegir entre configuraciones de 6, 7 u 8 plazas según tus necesidades.¿Por qué lo recomendamos?Ideal 

In [148]:
# Lo añado a la lista del DF
lista_caracteristicas_DF.extend(lista_caracteristicas_url_6_corregida)
len(lista_caracteristicas_DF)

107

In [149]:
# Otra página con monovolúmenes
url_7 = 'https://www.carwow.es/furgonetas/familiar#gref'
accesibilidad(url_7)

200 OK


In [150]:
#Lista vehiculos
lista_coches_url_7 = lista_coches(url_7, 'h3', class_="deal-review-highlight-automated-ctas__title")
lista_coches_DF.extend(lista_coches_url_7)
len(lista_coches_DF)

112

In [151]:
# Lista valoraciones
lista_valoraciones_url_7 = valoraciones(url_7, 'div', class_="wowscore-pill")
lista_valoraciones_DF.extend(lista_valoraciones_url_7)
len(lista_valoraciones_DF)

112

In [152]:
# Lista a favor
lista_lo_mejor_url_7 = lo_mejor(url_7, 'li', class_="list-icon__item list-icon__item--positive")
lista_a_favor_url_7 = agrupar_de_tres(lista_lo_mejor_url_7) # agrupo de tres en tres
lista_a_favor_url_7_corregida = [lista_a_favor_url_7[i] for i in range(0, len(lista_a_favor_url_7), 2)] # para quedarme sólo con una copia
lista_a_favor_DF.extend(lista_a_favor_url_7_corregida)
len(lista_a_favor_DF)

112

In [153]:
# Lista en contra
lista_lo_peor_url_7 = lo_mejor(url_7, 'li', class_="list-icon__item list-icon__item--negative")
lista_en_contra_url_7 = agrupar_de_tres(lista_lo_peor_url_7) # agrupo de tres en tres
lista_en_contra_url_7_corregida = [lista_en_contra_url_7[i] for i in range(0, len(lista_en_contra_url_7), 2)] # para quedarme sólo con una copia
lista_en_contra_DF.extend(lista_en_contra_url_7_corregida)
len(lista_en_contra_DF)

112

In [154]:
# Lista opinion
lista_opiniones_url_7 = opiniones(url_7, 'div', class_="deal-review-highlight-automated-ctas__body--no-read-more")
lista_opinion_DF.extend(lista_opiniones_url_7)
len(lista_opinion_DF)

112

In [155]:
# Lista precios
lista_precios_url_7= lista_precios(url_7, 'div', class_="div tile-button__subtitle")
lista_precios_url_7_corregida = [lista_precios_url_7[i] for i in range(0, len(lista_precios_url_7), 2)] # para quedarme sólo con una copia
lista_precios_DF.extend(lista_precios_url_7_corregida)
len(lista_precios_DF)

112

In [156]:
# Lista características (no tiene la web)
lista_caracteristicas_url_7 = crear_lista_none(5)
lista_caracteristicas_DF.extend(lista_caracteristicas_url_7)
len(lista_caracteristicas_DF)

112

In [157]:
# Octava url
url_8 = 'https://rentingcoches.com/mejores-coches-7-plazas'
accesibilidad(url_8)

200 OK


In [158]:
# Lista coches
lista_coches_url_8 = lista_coches(url_8, 'h3', None)
lista_coches_DF.extend(lista_coches_url_8)
len(lista_coches_DF)

120

In [159]:
# Lista valoraciones
lista_valoraciones_url_8 = crear_lista_none(8)
lista_valoraciones_DF.extend(lista_valoraciones_url_8)
len(lista_valoraciones_DF)

120

In [160]:
# Lista a favor
lista_a_favor_url_8 = lo_mejor (url_8, 'div', class_="highlighted-box") 
lista_a_favor_url_8 

['Lo mejorVersatilidad con siete plazas y diseño moderno',
 'Lo mejorableAsistencia eléctrica limitada',
 'Lo mejorTecnología e-POWER para una conducción suave y eficiente',
 'Lo mejorablePrecio inicial elevado en comparación con la competencia',
 'Lo mejorAmplio espacio interior y eficiencia en consumo',
 'Lo mejorablePrestaciones dinámicas que podrían ser más deportivas',
 'Lo mejorAmplio espacio interior y opciones de motorización eficientes',
 'Lo mejorableEl precio de la versión híbrida puede ser elevado',
 'Lo mejorAmplio espacio interior y eficiencia en consumo',
 'Lo mejorablePrestaciones dinámicas que podrían ser más deportivas',
 'Lo mejorConfort de marcha y equipamiento tecnológico avanzado',
 'Lo mejorableConsumo algo elevado',
 'Lo mejorVersatilidad en la configuración de asientos',
 'Lo mejorablePuede resultar menos manejable en entornos urbanos',
 'Lo mejorRelación calidad-precio excepcional',
 'Lo mejorableAcabados interiores demasiado sencillos']

In [161]:
# Aparecen todos los valores positivos y negativos
# Me quedo con los de valor positivo 
lista_lo_mejor_url_8 = [lista_a_favor_url_8[i] for i in range(0, len(lista_a_favor_url_8), 2)]
lista_a_favor_DF.extend(lista_lo_mejor_url_8)
len(lista_a_favor_DF)

120

In [162]:
# Lista en contra
# Aprovecho el listado obtenido anteriormente 
# Me quedo con los de valor negativo
lista_lo_peor_url_8 = [lista_a_favor_url_8[i] for i in range(1, len(lista_a_favor_url_8), 2)]
lista_en_contra_DF.extend(lista_lo_peor_url_8)
len(lista_en_contra_DF)

120

In [163]:
# Lista opiniones
lista_opninones_url_8 = opiniones (url_8,'p', class_="mb-0") 
lista_opinion_DF.extend(lista_opninones_url_8)
len(lista_opinion_DF)

120

In [164]:
# Lista precios
lista_precios_url_8 = lista_precios(url_8, 'p', class_="info")
lista_precios_DF.extend(lista_precios_url_8)
len(lista_precios_DF)

120

In [165]:
# Lista característcas
lista_caracteristicas_url_8 = caracteristicas(url_8, 'div', class_="d-flex flex-wrap")
lista_caracteristicas_DF.extend(lista_caracteristicas_url_8)
len(lista_caracteristicas_DF)

120

In [166]:
# Novena página (misma que antes pero con otras informaciones)
url_9 = 'https://rentingcoches.com/mejores-coches-hibridos/7-plazas'
accesibilidad(url_9)

200 OK


In [167]:
# Lista coches
lista_coches_url_9 = lista_coches(url_9, 'h3', None)
lista_coches_DF.extend(lista_coches_url_9)
print(len(lista_coches_DF))

# Lista valoraciones
lista_valoraciones_url_9 = crear_lista_none(7)
lista_valoraciones_DF.extend(lista_valoraciones_url_9)
print(len(lista_valoraciones_DF))

# Lista a favor
lista_a_favor_url_9 = lo_mejor (url_9, 'div', class_="highlighted-box") 
# Aparecen positivo y negativo
lista_lo_mejor_url_9 = [lista_a_favor_url_9[i] for i in range(0, len(lista_a_favor_url_9), 2)]
lista_a_favor_DF.extend(lista_lo_mejor_url_9)
print(len(lista_a_favor_DF))

# Lista en contra
# Aprovecho el listado obtenido anteriormente 
# Me quedo con los de valor positivo 
lista_lo_peor_url_9 = [lista_a_favor_url_9[i] for i in range(1, len(lista_a_favor_url_9), 2)]
lista_en_contra_DF.extend(lista_lo_peor_url_9)
print(len(lista_en_contra_DF))

# Lista opiniones
lista_opninones_url_9 = opiniones (url_9,'p', class_="mb-0") 
lista_opinion_DF.extend(lista_opninones_url_9)
print(len(lista_opinion_DF))

# Lista precios
lista_precios_url_9 = lista_precios(url_9, 'p', class_="info")
lista_precios_DF.extend(lista_precios_url_9)
print(len(lista_precios_DF))

# Lista característcas
lista_caracteristicas_url_9 = caracteristicas(url_9, 'div', class_="d-flex flex-wrap")
lista_caracteristicas_DF.extend(lista_caracteristicas_url_9)
print(len(lista_caracteristicas_DF))

127
127
127
127
127
127
127


In [168]:
# Url 10
url_10 = 'https://rentingcoches.com/mejores-coches-electricos/7-plazas'
accesibilidad(url_10)

200 OK


In [169]:
# Lista coches
lista_coches_url_10 = lista_coches(url_10, 'h3', None)
lista_coches_DF.extend(lista_coches_url_10)
print(len(lista_coches_DF))

# Lista valoraciones
lista_valoraciones_url_10 = crear_lista_none(5)
lista_valoraciones_DF.extend(lista_valoraciones_url_10)
print(len(lista_valoraciones_DF))

# Lista a favor
lista_a_favor_url_10 = lo_mejor (url_10, 'div', class_="highlighted-box") 
# Aparecen positivo y negativo
lista_lo_mejor_url_10 = [lista_a_favor_url_10[i] for i in range(0, len(lista_a_favor_url_10), 2)]
lista_a_favor_DF.extend(lista_lo_mejor_url_10)
print(len(lista_a_favor_DF))

# Lista en contra
# Aprovecho el listado obtenido anteriormente 
# Me quedo con los de valor positivo 
lista_lo_peor_url_10 = [lista_a_favor_url_10[i] for i in range(1, len(lista_a_favor_url_10), 2)]
lista_en_contra_DF.extend(lista_lo_peor_url_10)
print(len(lista_en_contra_DF))

# Lista opiniones
lista_opninones_url_10 = opiniones (url_10,'p', class_="mb-0") 
lista_opinion_DF.extend(lista_opninones_url_10)
print(len(lista_opinion_DF))

# Lista precios
lista_precios_url_10 = lista_precios(url_10, 'p', class_="info")
lista_precios_DF.extend(lista_precios_url_10)
print(len(lista_precios_DF))

# Lista característcas
lista_caracteristicas_url_10 = caracteristicas(url_10, 'div', class_="d-flex flex-wrap")
lista_caracteristicas_DF.extend(lista_caracteristicas_url_10)
print(len(lista_caracteristicas_DF))

132
132
132
132
132
132
132


In [170]:
# Url 11
url_11= 'https://www.carwow.es/coches-7-plazas#gref'
accesibilidad(url_11)

200 OK


In [171]:
#Lista vehiculos
lista_coches_url_11 = lista_coches(url_11, 'h3', class_="deal-review-highlight-automated-ctas__title")
lista_coches_DF.extend(lista_coches_url_11)
print(len(lista_coches_DF))

# Lista valoraciones
lista_valoraciones_url_11 = valoraciones(url_11, 'div', class_="wowscore-pill")
lista_valoraciones_DF.extend(lista_valoraciones_url_11)
print(len(lista_valoraciones_DF))

# Lista a favor
lista_lo_mejor_url_11 = lo_mejor(url_11, 'li', class_="list-icon__item list-icon__item--positive")
lista_a_favor_url_11 = agrupar_de_tres(lista_lo_mejor_url_11) # agrupo de tres en tres
lista_a_favor_url_11_corregida = [lista_a_favor_url_11[i] for i in range(0, len(lista_a_favor_url_11), 2)] # para quedarme sólo con una copia
lista_a_favor_DF.extend(lista_a_favor_url_11_corregida)
print(len(lista_a_favor_DF))

# Lista en contra
lista_lo_peor_url_11 = lo_mejor(url_11, 'li', class_="list-icon__item list-icon__item--negative")
lista_en_contra_url_11 = agrupar_de_tres(lista_lo_peor_url_11) # agrupo de tres en tres
lista_en_contra_url_11_corregida = [lista_en_contra_url_11[i] for i in range(0, len(lista_en_contra_url_11), 2)] # para quedarme sólo con una copia
lista_en_contra_DF.extend(lista_en_contra_url_11_corregida)
print(len(lista_en_contra_DF))

# Lista opinion
lista_opiniones_url_11 = opiniones(url_11, 'div', class_="deal-review-highlight-automated-ctas__body--no-read-more")
lista_opinion_DF.extend(lista_opiniones_url_11)
print(len(lista_opinion_DF))

#Lista precios
lista_precios_url_11= lista_precios(url_11, 'div', class_="div tile-button__subtitle")
lista_precios_url_11_corregida = [lista_precios_url_11[i] for i in range(0, len(lista_precios_url_11), 2)] # para quedarme sólo con una copia
lista_precios_DF.extend(lista_precios_url_11_corregida)
print(len(lista_precios_DF))

# Lista características (no tiene la web)
lista_caracteristicas_url_11 = crear_lista_none(10)
lista_caracteristicas_DF.extend(lista_caracteristicas_url_11)
print(len(lista_caracteristicas_DF))

142
142
142
142
142
142
142


In [172]:
# Url 12
url_12 ='https://www.diariomotor.com/guias-compra/coches-7-plazas-mas-baratos/'
accesibilidad(url_12)

200 OK


In [173]:
# Lista de coches (extraigo e inserto a la lista del DF)
lista_coches_url_12 = lista_coches(url_12, 'h2', class_="m-0 text-3xl leading-none -tracking-xs md:-tracking-s md:text-4xl")
lista_coches_DF.extend(lista_coches_url_12)
print(len(lista_coches_DF)) 

# Lista Valoraciones (extraigo e inserto en la lista del DF)
lista_valoraciones_url_12 = valoraciones(url_12, 'div', class_="mt-1 leading-7 numero")
lista_valoraciones_DF.extend(lista_valoraciones_url_12)
print(len(lista_valoraciones_DF))

# Lista a favor (no hay datos en esta página)
# Genero una lista con valores nulos
lista_a_favor_url_12 = crear_lista_none(10)
lista_a_favor_DF.extend(lista_a_favor_url_12)
print(len(lista_a_favor_DF))

# Lista en contra (no hay datos en esta página)
# Genero una lista con valores nulos
lista_en_contra_url_12 = crear_lista_none(10)   
lista_en_contra_DF.extend(lista_en_contra_url_12)
print(len(lista_en_contra_DF))

# Opiniones (extraigo datos y agrego a la lista del DF)
lista_opiniones_url_12 = opiniones(url_12, 'div',  class_="py-4 border-b border-gray-100 cuerpo")
lista_opinion_DF.extend(lista_opiniones_url_12)
print(len(lista_opinion_DF))

# Precios (extraigo datos y agrego a la lista del DF)
lista_precios_url_12 = lista_precios(url_12, 'div', class_="flex flex-col text-3xl font-semibold leading-7 text-main md:text-4xl -tracking-xs md:mt-0 md:mr-2")
lista_precios_DF.extend(lista_precios_url_12)
print(len(lista_precios_DF))

# Características
# En esta ocasión se hace en dos partes: extraemos etiquetas y resto características
etiquetas_url_12 = lista_etiquetas(url_12)
lista_etiquetas_url_12 = etiquetas_url_12[0]
lista_etiquetas_url_12_corregida = []
lista_etiquetas_url_12_corregida.append(agregar_un_parrafo(lista_etiquetas_url_12, 0))
lista_etiquetas_url_12_corregida.append(agregar_dos_parrafos(lista_etiquetas_url_12, 1,2))
lista_etiquetas_url_12_corregida.append(agregar_dos_parrafos(lista_etiquetas_url_12, 3,4))
lista_etiquetas_url_12_corregida.append(agregar_un_parrafo(lista_etiquetas_url_12, 5))
lista_etiquetas_url_12_corregida.append(agregar_un_parrafo(lista_etiquetas_url_12, 6))
lista_etiquetas_url_12_corregida.append(agregar_un_parrafo(lista_etiquetas_url_12, 7))
lista_etiquetas_url_12_corregida.append(agregar_un_parrafo(lista_etiquetas_url_12, 8))
lista_etiquetas_url_12_corregida.append(agregar_un_parrafo(lista_etiquetas_url_12, 9))
lista_etiquetas_url_12_corregida.append(agregar_un_parrafo(lista_etiquetas_url_12, 10))
lista_etiquetas_url_12_corregida.insert(2,['']) # Introduzco el valor Nulo que falta
# Extraigo resto de características
lista_caracteristicas_url_12 = caracteristicas (url_12, 'div', class_="flex flex-col text-base font-semibold leading-4 text-gray-700 uppercase texto -tracking-xs")
lista_caracteristicas_url_12_corregida = agrupar_de_tres(lista_caracteristicas_url_12)
# A continuación debo unir las dos listas, pero que se combinen, para que generen una sola todos los elementos
lista_caracteristicas_url_12_completa = []
for indice, item in enumerate(lista_caracteristicas_url_12_corregida):
    lista_caracteristicas_url_12_completa.append(item+lista_etiquetas_url_12_corregida[indice])
print(lista_caracteristicas_url_12_completa)
# Finalmente la añado a la lista del DF
lista_caracteristicas_DF.extend(lista_caracteristicas_url_12_completa)
len(lista_caracteristicas_DF)

152
152
152
152
152
152
[['Potencia 110 CV', 'Consumo 7.20-7.20 l/100Km', 'Maletero', 'Etiqueta ambiental C'], ['Potencia 100-110 CV', 'Consumo 5.80-7.70 l/100Km', 'Maletero 595', ['Etiqueta ambiental CEtiqueta ambiental ECO']], ['Potencia CV', 'Consumo l/100Km', 'Maletero 460', ''], ['Potencia 100-145 CV', 'Consumo 5.30-6.20 l/100Km', 'Maletero 460', ['Etiqueta ambiental CEtiqueta ambiental ECO']], ['Potencia 139 CV', 'Consumo 8.50-8.50 l/100Km', 'Maletero', 'Etiqueta ambiental C'], ['Potencia 102-122 CV', 'Consumo 5.40-7.00 l/100Km', 'Maletero', 'Etiqueta ambiental C'], ['Potencia 110-146 CV', 'Consumo 5.20-5.20 l/100Km', 'Maletero 460', 'Etiqueta ambiental ECO'], ['Potencia 149 CV', 'Consumo 9.90-9.90 l/100Km', 'Maletero', 'Etiqueta ambiental C'], ['Potencia 140 CV', 'Consumo 4.80-4.90 l/100Km', 'Maletero 595', 'Etiqueta ambiental ECO'], ['Potencia 177 CV', 'Consumo l/100Km', 'Maletero 375', 'Etiqueta ambiental C']]


152

In [174]:
# Url 13 (no tiene demasiada información, pero me interesan nombres y opiniones)
url_13 = 'https://www.coches.com/noticias/informes/guia-de-compra-monovolumenes-grandes/119443'
accesibilidad (url_13)

200 OK


In [175]:
# Lista de coches (extraigo e inserto a la lista del DF)
lista_coches_url_13 = lista_coches(url_13, 'h2', None)
del lista_coches_url_13 [:2] # Borro entradas que no me interesan
del lista_coches_url_13 [-3:]
lista_coches_DF.extend(lista_coches_url_13)
print(len(lista_coches_DF)) 

# Lista Valoraciones (no tiene esta web)
lista_valoraciones_url_13 = crear_lista_none(13)
lista_valoraciones_DF.extend(lista_valoraciones_url_13)
print(len(lista_valoraciones_DF))

# Lista a favor (no tiene esta web)
lista_a_favor_url_13 = crear_lista_none(13)
lista_a_favor_DF.extend(lista_a_favor_url_13)
print(len(lista_a_favor_DF))

# Lista a favor (no tiene esta web)
lista_en_contra_url_13 = crear_lista_none(13)
lista_en_contra_DF.extend(lista_en_contra_url_13)
print(len(lista_en_contra_DF))

# Opiniones (extraigo datos y agrego a la lista del DF)
lista_opiniones_url_13 = opiniones(url_13, 'p', None)
del lista_opiniones_url_13 [:13]
del lista_opiniones_url_13 [-7:]
introducir_nulos(lista_opiniones_url_13, (47,59,65,71,77))
lista_opiniones_url_13_agrupada =[lista_opiniones_url_13[i:i+6] for i in range(0, len(lista_opiniones_url_13), 6)]
lista_opinion_DF.extend(lista_opiniones_url_13_agrupada)
print(len(lista_opinion_DF))

# Precios (no vienen precios)
lista_precios_url_13 = crear_lista_none(13)
lista_precios_DF.extend(lista_precios_url_13)
print(len(lista_precios_DF))

# Características (no vienen precios)
lista_caracteristicas_url_13 = crear_lista_none(13)
lista_caracteristicas_DF.extend(lista_caracteristicas_url_13)
print(len(lista_caracteristicas_DF))

165
165
165
165
['El Citroën SpaceTourer se actualizó en 2024 y recuperó una versión de combustión, que la marca había descatalogado en los últimos compases en que estuvo en venta el anterior modelo. Se ofrece con dos longitudes de carrocería (4,98 o 5,33 metros). Puede diferenciarse por los cambios en el diseño de la parte frontal y, sobre todo, ganó equipamiento.', 'Este monovolumen cuenta con un amplio habitáculo en el que pueden viajar hasta nueve pasajeros puede elegirse con diversas cofiguraciuones interiores. De serie cuenta con ocho plazas distribuidas en una configuración 2+3+3 pero es posible elegir una versión muy lujosa con la segunda fila de asientos mira hacia atrás y capacidad reducida a siete personas.', 'En lo que respecta al apartado mecánico, la versión elegida es el 2.0 BlueHDI diésel de 180 CV, asociado a la caja de cambios automática EAT8 de ocho velocidades y un sistema de tracción delantera. Es un conjunto con el que realizamos una prueba a fondo en la anterior 

In [176]:
# Url 14 (no tiene mucha informacion pero me interesan los nombres y las caracteristicas)
url_14 = 'https://carnovo.com/es/guias/mejores-furgonetas/'
accesibilidad(url_14)

200 OK


In [177]:
# Lista de coches
lista_coches_url_14 = lista_coches(url_14, 'h3', None)
del lista_coches_url_14[6]
del lista_coches_url_14[-2:]
lista_coches_DF.extend(lista_coches_url_14)
print(len(lista_coches_DF))

# Lista Valoraciones (no tiene esta web)
lista_valoraciones_url_14 = crear_lista_none(7)
lista_valoraciones_DF.extend(lista_valoraciones_url_14)
print(len(lista_valoraciones_DF))

# Lista a favor (no tiene esta web)
lista_a_favor_url_14 = crear_lista_none(7)
lista_a_favor_DF.extend(lista_a_favor_url_14)
print(len(lista_a_favor_DF))

# Lista a favor (no tiene esta web)
lista_en_contra_url_14 = crear_lista_none(7)
lista_en_contra_DF.extend(lista_en_contra_url_14)
print(len(lista_en_contra_DF))

# Opiniones (extraigo datos y agrego a la lista del DF)
lista_opiniones_url_14 = crear_lista_none(7)
lista_opinion_DF.extend(lista_opiniones_url_14)
print(len(lista_opinion_DF))

# Precios (no vienen precios)
lista_precios_url_14 = crear_lista_none(7)
lista_precios_DF.extend(lista_precios_url_14)
print(len(lista_precios_DF))

# Características (no vienen precios)
lista_caracteristicas_url_14 = opiniones(url_14, 'p', None)
lista_caracteristicas_url_14_corregida = []
lista_caracteristicas_url_14_corregida.append(agregar_un_parrafo(lista_caracteristicas_url_14, 2))
lista_caracteristicas_url_14_corregida.append(agregar_un_parrafo(lista_caracteristicas_url_14, 4))
lista_caracteristicas_url_14_corregida.append(agregar_un_parrafo(lista_caracteristicas_url_14, 6))
lista_caracteristicas_url_14_corregida.append(agregar_un_parrafo(lista_caracteristicas_url_14, 8))
lista_caracteristicas_url_14_corregida.append(agregar_un_parrafo(lista_caracteristicas_url_14, 10))
lista_caracteristicas_url_14_corregida.append(agregar_un_parrafo(lista_caracteristicas_url_14, 12))
lista_caracteristicas_url_14_corregida.append(agregar_un_parrafo(lista_caracteristicas_url_14, 14))
lista_caracteristicas_DF.extend(lista_caracteristicas_url_14_corregida)
print(len(lista_caracteristicas_DF))

172
172
172
172
172
172
172


In [178]:
df_extraccion = pd.DataFrame.from_dict(diccionario_coches)
df_extraccion

,Nombre,Valoraciones,A favor,En contra,Opiniones,Precio,Características
0,1. Mazda CX-80,9/10,[Excelente eficiencia tanto en el PHEV como en...,[La suspensión puede sentirse algo seca en cie...,El CX-80 usa la misma plataforma y motores que...,Desde 54.668 €,None
1,2. Renault Espace,9/10,"[Consumo, Equipamiento disponible, Opción de s...","[Funcionamiento poco refinado en ocasiones, Ga...",El nuevo Renault Espace es ahora un SUV con un...,Desde 38.280 €,None
2,3. Hyundai Santa Fe,8/10,[Equipamiento y tecnología propios de coches d...,"[Precio elevado, Escasa autonomía eléctrica de...",El Santa Fe dispone de motorizaciones híbrida ...,Desde 51.478 €,None
3,4. Skoda Kodiaq,8/10,"[Practicidad y espacio disponible, Oferta mecá...","[Ergonomía de algunos mandos, Plazas de la ter...","El Skoda Kodiaq es mucho más barato que el Q7,...",Desde 40.001 €,None
4,5. Peugeot 5008,8/10,"[Amplio interior con 7 plazas de serie, Comodi...",[La calidad del sistema multimedia podría mejo...,El Peugeot 5008 es un coche muy redondo y cohe...,Desde 34.885 €,None
...,...,...,...,...,...,...,...
167,Opel Combo e-life,None,None,None,None,None,"[Disponible en dos tallas: Talla L y Talla XL,..."
168,Mercedes Clase T,None,None,None,None,None,[El Mercedes Clase T es un monovolumen compact...
169,Nissan Townstar,None,None,None,None,None,[El Nissan Townstar es una furgoneta ligera qu...
170,Peugeot e-rifter,None,None,None,None,None,"[Disponible con los acabados Active Pack, Allu..."


In [179]:
# Lo guardo e formato csv para poder trabajar con él en otro notebbok
df_extraccion.to_csv('DF_extraccion.csv', index=False)

In [180]:
lista_urls = [] # Convertirlo a diccionario
lista_urls.append(url_1)
lista_urls.append(url_2)
lista_urls.append(url_3)
lista_urls.append(url_4)
lista_urls.append(url_5)
lista_urls.append(url_6)
lista_urls.append(url_7)
lista_urls.append(url_8)
lista_urls.append(url_9)
lista_urls.append(url_10)
lista_urls.append(url_11)
lista_urls.append(url_12)
lista_urls.append(url_13)
lista_urls.append(url_14)
lista_urls



['https://www.carwow.es/suvs/7-plazas?utm_source=google&utm_medium=cpc&utm_account=1067952725&utm_campaign=16616870776&utm_group=140354601048&utm_keyword=&device=c&campaignid=16616870776&adgroupid=140354601048&gad_source=1&gbraid=0AAAAADkNbBUPhyVGM4sZxbLeGiQKHZ1nw&gclid=CjwKCAjwq7fABhB2EiwAwk-YbDwojWaxN2WG0_cnvz9JYpB20AtA5WJy8TP3udES_xm-XQdCLAp3QRoCxhMQAvD_BwE',
 'https://www.autohero.com/es/consejos/elegir/mejor-coche/suv-7-plazas/',
 'http://quecochemecompro.com/blog/comparativa-suv-7-plazas-guia-de-compra/',
 'https://www.diariomotor.com/guias-compra/mejores-suv-7-plazas/',
 'https://www.quecochemecompro.com/blog/coches-de-7-plazas-todos-los-modelos/',
 'https://www.autohero.com/es/consejos/elegir/mejor-coche/monovolumen-7-plazas/',
 'https://www.carwow.es/furgonetas/familiar#gref',
 'https://rentingcoches.com/mejores-coches-7-plazas',
 'https://rentingcoches.com/mejores-coches-hibridos/7-plazas',
 'https://rentingcoches.com/mejores-coches-electricos/7-plazas',
 'https://www.carwow.